# 🎯 SHAP and XAI: Comprehensive Hands-on Tutorial## Model Explainability - From Theory to Practice**Based on Lecture 20: Model Explainability - SHAP and Deep Learning XAI**---### 📚 Learning ObjectivesBy the end of this tutorial, you will be able to:1. ✅ Understand Shapley values from game theory and their application to ML2. ✅ Implement different SHAP explainers (Tree, Kernel, Deep, Gradient)3. ✅ Create and interpret various SHAP visualizations4. ✅ Apply SHAP to real-world problems (credit, healthcare, images)5. ✅ Use advanced XAI techniques (Attention, Grad-CAM, Integrated Gradients)**Duration**: ~2-3 hours  **Level**: Intermediate  **Prerequisites**: Basic ML knowledge, scikit-learn experience**Instructor**: Ho-min Park (homin.park@ghent.ac.kr)---### 📖 Tutorial Structure| Part | Topic | Exercises | Time ||------|-------|-----------|------|| **0** | Setup & Environment | - | 5 min || **1** | SHAP Fundamentals | 4 exercises | 30 min || **2** | SHAP Implementation Methods | 5 exercises | 45 min || **3** | SHAP Visualizations | 4 exercises | 30 min || **4** | Advanced XAI Techniques | 3 exercises | 40 min || **5** | Real-world Application | 1 project | 20 min |---### 💡 Tips for Learning- 🔄 Run each cell sequentially- ✏️ Complete "Your Turn" exercises- 🤔 Read interpretation guides carefully- 💬 Ask questions when stuck- 🎨 Experiment with visualizations

---# 📦 Part 0: Setup & EnvironmentLet's prepare our workspace by installing necessary libraries and loading datasets.

In [ ]:
# Installation (uncomment if needed)# !pip install shap scikit-learn xgboost matplotlib seaborn pandas numpy plotlyprint('✅ Installation ready!')

In [ ]:
# Import core librariesimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as snsimport warningswarnings.filterwarnings('ignore')# Machine Learningfrom sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, GradientBoostingRegressorfrom sklearn.tree import DecisionTreeClassifierfrom sklearn.svm import SVCfrom sklearn.neural_network import MLPClassifierfrom sklearn.model_selection import train_test_splitfrom sklearn.metrics import accuracy_score, mean_squared_error, classification_reportfrom sklearn.datasets import load_iris, load_breast_cancer, load_winefrom sklearn.preprocessing import StandardScaler# SHAPimport shap# Advanced visualizationimport plotly.graph_objects as goimport plotly.express as px# Stylingplt.style.use('seaborn-v0_8-whitegrid')sns.set_palette("husl")# ConstantsRANDOM_STATE = 42np.random.seed(RANDOM_STATE)print(f'✅ Libraries imported successfully!')print(f'📊 SHAP version: {shap.__version__}')print(f'🐍 NumPy version: {np.__version__}')print(f'🐼 Pandas version: {pd.__version__}')

In [ ]:
# Load and prepare datasetsprint('=' * 60)print('📁 LOADING DATASETS')print('=' * 60)# 1. Iris Dataset (Classification)iris = load_iris(as_frame=True)iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)iris_df['target'] = iris.targetiris_df['species'] = iris_df['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})print(f'\n1️⃣  Iris Dataset: {iris_df.shape}')print(f'   Features: {list(iris.feature_names)}')print(f'   Classes: {iris.target_names}')# 2. Breast Cancer Dataset (Binary Classification)cancer = load_breast_cancer(as_frame=True)cancer_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)cancer_df['target'] = cancer.targetcancer_df['diagnosis'] = cancer_df['target'].map({0: 'malignant', 1: 'benign'})print(f'\n2️⃣  Breast Cancer Dataset: {cancer_df.shape}')print(f'   Features: {len(cancer.feature_names)} features')print(f'   Classes: Malignant ({(cancer_df.target==0).sum()}), Benign ({(cancer_df.target==1).sum()})')# 3. Wine Dataset (Multi-class)wine = load_wine(as_frame=True)wine_df = pd.DataFrame(wine.data, columns=wine.feature_names)wine_df['target'] = wine.targetprint(f'\n3️⃣  Wine Dataset: {wine_df.shape}')print(f'   Features: {len(wine.feature_names)} features')print(f'   Classes: {len(wine.target_names)} wine types')# 4. Synthetic Credit Approval Datasetn = 1000np.random.seed(RANDOM_STATE)credit_df = pd.DataFrame({    'Age': np.random.randint(22, 70, n),    'Income': np.random.randint(20000, 150000, n),    'Credit_Score': np.random.randint(300, 850, n),    'Debt_Ratio': np.random.uniform(0, 0.6, n),    'Employment_Years': np.random.randint(0, 40, n),    'Previous_Defaults': np.random.binomial(3, 0.1, n),    'Loan_Amount': np.random.randint(5000, 50000, n)})# Create approval target based on realistic criteriascore = (    (credit_df['Income'] / 50000) * 0.25 +    (credit_df['Credit_Score'] / 850) * 0.35 +    (1 - credit_df['Debt_Ratio']) * 0.20 +    (credit_df['Employment_Years'] / 40) * 0.10 +    (1 - credit_df['Previous_Defaults'] / 3) * 0.10)credit_df['Approved'] = (score + np.random.normal(0, 0.15, n) > 0.5).astype(int)print(f'\n4️⃣  Credit Approval Dataset: {credit_df.shape}')print(f'   Approval Rate: {credit_df["Approved"].mean():.1%}')print(f'   Features: {list(credit_df.columns[:-1])}')# 5. Synthetic Housing Price Dataset (Regression)n_houses = 500np.random.seed(RANDOM_STATE)housing_df = pd.DataFrame({    'Square_Feet': np.random.randint(800, 4000, n_houses),    'Bedrooms': np.random.randint(1, 6, n_houses),    'Bathrooms': np.random.randint(1, 4, n_houses),    'Age': np.random.randint(0, 50, n_houses),    'Distance_to_City': np.random.uniform(1, 30, n_houses),    'School_Rating': np.random.randint(1, 11, n_houses)})# Price formula with non-linear relationshipsbase_price = 100000housing_df['Price'] = (    base_price +    housing_df['Square_Feet'] * 150 +    housing_df['Bedrooms'] * 20000 +    housing_df['Bathrooms'] * 15000 -    housing_df['Age'] * 2000 -    housing_df['Distance_to_City'] * 3000 +    housing_df['School_Rating'] * 10000 +    np.random.normal(0, 20000, n_houses))print(f'\n5️⃣  Housing Price Dataset: {housing_df.shape}')print(f'   Price Range: ${housing_df["Price"].min():,.0f} - ${housing_df["Price"].max():,.0f}')print(f'   Mean Price: ${housing_df["Price"].mean():,.0f}')print('\n' + '=' * 60)print('✅ All datasets loaded successfully!')print('=' * 60)

In [ ]:
# Visualize datasetsfig, axes = plt.subplots(2, 3, figsize=(18, 10))fig.suptitle('📊 Dataset Overview', fontsize=16, fontweight='bold', y=1.02)# 1. Irisiris_df.groupby('species').size().plot(kind='bar', ax=axes[0, 0], color='skyblue', edgecolor='black')axes[0, 0].set_title('Iris: Class Distribution', fontweight='bold')axes[0, 0].set_xlabel('Species')axes[0, 0].set_ylabel('Count')axes[0, 0].tick_params(axis='x', rotation=45)# 2. Cancercancer_df.groupby('diagnosis').size().plot(kind='bar', ax=axes[0, 1], color=['salmon', 'lightgreen'], edgecolor='black')axes[0, 1].set_title('Breast Cancer: Diagnosis Distribution', fontweight='bold')axes[0, 1].set_xlabel('Diagnosis')axes[0, 1].set_ylabel('Count')axes[0, 1].tick_params(axis='x', rotation=45)# 3. Creditcredit_df['Approved'].value_counts().plot(kind='pie', ax=axes[0, 2], autopct='%1.1f%%',                                           labels=['Rejected', 'Approved'], colors=['lightcoral', 'lightblue'])axes[0, 2].set_title('Credit: Approval Rate', fontweight='bold')axes[0, 2].set_ylabel('')# 4. Credit Score Distributionaxes[1, 0].hist(credit_df['Credit_Score'], bins=30, color='purple', alpha=0.7, edgecolor='black')axes[1, 0].set_title('Credit: Score Distribution', fontweight='bold')axes[1, 0].set_xlabel('Credit Score')axes[1, 0].set_ylabel('Frequency')# 5. Housing Price Distributionaxes[1, 1].hist(housing_df['Price']/1000, bins=30, color='orange', alpha=0.7, edgecolor='black')axes[1, 1].set_title('Housing: Price Distribution', fontweight='bold')axes[1, 1].set_xlabel('Price ($1000s)')axes[1, 1].set_ylabel('Frequency')# 6. Housing: Price vs Square Feetscatter = axes[1, 2].scatter(housing_df['Square_Feet'], housing_df['Price']/1000,                              c=housing_df['School_Rating'], cmap='viridis', alpha=0.6, s=30)axes[1, 2].set_title('Housing: Price vs Size', fontweight='bold')axes[1, 2].set_xlabel('Square Feet')axes[1, 2].set_ylabel('Price ($1000s)')plt.colorbar(scatter, ax=axes[1, 2], label='School Rating')plt.tight_layout()plt.show()print('\n📈 Dataset visualizations completed!')

---# 📊 Part 1: SHAP Fundamentals## Understanding Shapley Values from Game TheorySHAP (SHapley Additive exPlanations) is based on **cooperative game theory**, specifically Shapley values introduced by Lloyd Shapley in 1953.### 🎮 The Core IdeaImagine a team wins a game. **How do we fairly distribute credit among players?**Shapley values answer this by considering:- Each player's **marginal contribution** in all possible team combinations- Fair distribution that satisfies key properties### 🧮 Mathematical FoundationThe prediction can be decomposed as:```f(x) = φ₀ + φ₁ + φ₂ + ... + φₙ```Where:- `f(x)` = Model prediction for instance x- `φ₀` = Base value (expected prediction, usually mean)- `φᵢ` = SHAP value for feature i (contribution to prediction)### 📐 Shapley Value FormulaFor feature i:```φᵢ = Σ [|S|! × (n-|S|-1)! / n!] × [f(S ∪ {i}) - f(S)]```Where:- S = subset of features (coalition)- n = total number of features- The sum is over all possible subsets S not containing i### ✨ Key Properties1. **Local Accuracy**: Sum of SHAP values equals prediction - base value2. **Missingness**: Features not in the model have zero contribution3. **Consistency**: If a feature's contribution increases, its SHAP value doesn't decrease4. **Efficiency**: Sum of all contributions equals total "game value"5. **Symmetry**: Features with identical contributions get equal SHAP values---

## Exercise 1: Manual Shapley Value Calculation### 📖 ConceptLet's manually calculate Shapley values for a **house price prediction** with only 2 features:- 🏠 **Square Feet**- 📍 **Location Score**We'll explore all possible feature coalitions to understand the marginal contribution of each feature.

In [ ]:
# House price predictions for different feature combinationspredictions = {    'Empty (baseline)': 250,           # No features → base prediction    'Square_Feet only': 300,            # +50K from adding square feet    'Location only': 280,               # +30K from adding location    'Square_Feet + Location': 340      # Both features together}print('🏠 HOUSE PRICE PREDICTIONS')print('=' * 50)for coalition, price in predictions.items():    print(f'{coalition:25} → ${price}K')print('=' * 50)

In [ ]:
# Calculate Shapley value for Square Feetprint('\n📊 SHAPLEY VALUE CALCULATION: SQUARE FEET')print('=' * 60)# Path 1: Add Square_Feet to empty setcontribution_1 = predictions['Square_Feet only'] - predictions['Empty (baseline)']print(f'\nPath 1: Empty → Square_Feet')print(f'  Contribution: ${predictions["Square_Feet only"]}K - ${predictions["Empty (baseline)"]}K = ${contribution_1}K')# Path 2: Add Square_Feet to {Location}contribution_2 = predictions['Square_Feet + Location'] - predictions['Location only']print(f'\nPath 2: Location → Square_Feet + Location')print(f'  Contribution: ${predictions["Square_Feet + Location"]}K - ${predictions["Location only"]}K = ${contribution_2}K')# Shapley value (average of all paths)shap_sqft = (contribution_1 + contribution_2) / 2print(f'\n{"="*60}')print(f'✨ Shapley Value (Square Feet) = (${contribution_1}K + ${contribution_2}K) / 2')print(f'✨ Shapley Value (Square Feet) = ${shap_sqft}K')print(f'{"="*60}')

In [ ]:
# Calculate Shapley value for Locationprint('\n📊 SHAPLEY VALUE CALCULATION: LOCATION')print('=' * 60)# Path 1: Add Location to empty setcontribution_1_loc = predictions['Location only'] - predictions['Empty (baseline)']print(f'\nPath 1: Empty → Location')print(f'  Contribution: ${predictions["Location only"]}K - ${predictions["Empty (baseline)"]}K = ${contribution_1_loc}K')# Path 2: Add Location to {Square_Feet}contribution_2_loc = predictions['Square_Feet + Location'] - predictions['Square_Feet only']print(f'\nPath 2: Square_Feet → Square_Feet + Location')print(f'  Contribution: ${predictions["Square_Feet + Location"]}K - ${predictions["Square_Feet only"]}K = ${contribution_2_loc}K')# Shapley value (average of all paths)shap_location = (contribution_1_loc + contribution_2_loc) / 2print(f'\n{"="*60}')print(f'✨ Shapley Value (Location) = (${contribution_1_loc}K + ${contribution_2_loc}K) / 2')print(f'✨ Shapley Value (Location) = ${shap_location}K')print(f'{"="*60}')

In [ ]:
# Verify additive propertyprint('\n🔍 VERIFICATION: Additive Property')print('=' * 60)base_value = predictions['Empty (baseline)']final_prediction = predictions['Square_Feet + Location']print(f'Base Value: ${base_value}K')print(f'+ Square Feet contribution: ${shap_sqft}K')print(f'+ Location contribution: ${shap_location}K')print(f'{"-"*60}')calculated_prediction = base_value + shap_sqft + shap_locationprint(f'Calculated: ${calculated_prediction}K')print(f'Actual: ${final_prediction}K')if abs(calculated_prediction - final_prediction) < 0.01:    print('\n✅ VERIFIED! Shapley values satisfy the additive property.')else:    print('\n❌ Error in calculation!')print('=' * 60)

In [ ]:
# Visualize SHAP decompositionfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))# Left plot: Waterfall visualizationfeatures = ['Base', 'Square\nFeet', 'Location', 'Final\nPrediction']values = [base_value, shap_sqft, shap_location, final_prediction]colors = ['lightblue', 'green', 'green', 'darkblue']bars = ax1.bar(features, values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)for i, (feat, val) in enumerate(zip(features, values)):    ax1.text(i, val + 5, f'${val:.0f}K', ha='center', va='bottom',              fontweight='bold', fontsize=11)ax1.set_ylabel('Value ($K)', fontweight='bold', fontsize=12)ax1.set_title('🎯 SHAP Value Decomposition', fontweight='bold', fontsize=14)ax1.grid(axis='y', alpha=0.3)ax1.set_ylim(0, max(values) * 1.15)# Right plot: Contribution breakdowncontributions = [shap_sqft, shap_location]feature_names = ['Square Feet', 'Location']colors_pie = ['#2ecc71', '#3498db']wedges, texts, autotexts = ax2.pie(contributions, labels=feature_names, autopct='%1.1f%%',                                     colors=colors_pie, startangle=90, textprops={'fontsize': 11})for autotext in autotexts:    autotext.set_color('white')    autotext.set_fontweight('bold')ax2.set_title('🥧 Feature Contribution Breakdown', fontweight='bold', fontsize=14)plt.tight_layout()plt.show()print('\n💡 Interpretation:')print(f'  • Square Feet contributes ${shap_sqft}K ({shap_sqft/final_prediction*100:.1f}% of prediction)')print(f'  • Location contributes ${shap_location}K ({shap_location/final_prediction*100:.1f}% of prediction)')print(f'  • Base prediction (no info) is ${base_value}K')

### ✏️ Your Turn: Exercise 1a**Task**: Calculate Shapley values for a scenario with 3 features!Given predictions:```Empty: $200KA only: $250KB only: $230KC only: $240KA+B: $290KA+C: $300KB+C: $280KA+B+C: $340K```**Questions**:1. Calculate the Shapley value for feature A2. Verify the additive property3. Which feature is most important?**Hint**: For 3 features, each feature appears in 4 different coalitions (size 0, 1, 2, and 3).

In [ ]:
# YOUR CODE HERE# TODO: Calculate Shapley values for 3-feature scenariopredictions_3feat = {    'empty': 200,    'A': 250,    'B': 230,    'C': 240,    'AB': 290,    'AC': 300,    'BC': 280,    'ABC': 340}# Step 1: List all coalitions that DON'T contain A# Step 2: For each coalition, calculate A's marginal contribution# Step 3: Weight by coalition size probability# Step 4: Sum all weighted contributions# Example for Feature A:# Coalitions without A: {}, {B}, {C}, {B,C}print('Feature A Shapley Calculation:')print('Coalition | Contribution | Weight')print('-' * 40)# Your calculation here...

---## Exercise 2: SHAP vs LIME Comparison### 📖 Concept**LIME** (Local Interpretable Model-agnostic Explanations) and **SHAP** are both model-agnostic explanation methods, but they differ fundamentally:| Aspect | LIME | SHAP ||--------|------|------|| **Foundation** | Local linear approximation | Game theory (Shapley values) || **Sampling** | Perturbs instance randomly | Considers all coalitions || **Properties** | No theoretical guarantees | Satisfies axioms (efficiency, symmetry, etc.) || **Consistency** | May give different results | Consistent across runs || **Speed** | Faster | Slower (but optimized) || **Additivity** | Not guaranteed | Always additive |Let's compare them empirically!

In [ ]:
# Train a model on credit dataprint('🎓 Training Credit Approval Model...')print('=' * 60)X_credit = credit_df.drop(['Approved'], axis=1)y_credit = credit_df['Approved']X_train, X_test, y_train, y_test = train_test_split(    X_credit, y_credit, test_size=0.2, random_state=RANDOM_STATE, stratify=y_credit)# Train Random Forestrf_credit = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=RANDOM_STATE)rf_credit.fit(X_train, y_train)train_acc = accuracy_score(y_train, rf_credit.predict(X_train))test_acc = accuracy_score(y_test, rf_credit.predict(X_test))print(f'✅ Model trained successfully!')print(f'   Training Accuracy: {train_acc:.3f}')print(f'   Test Accuracy: {test_acc:.3f}')print('=' * 60)

In [ ]:
# SHAP Explanationprint('\n🔍 Generating SHAP Explanations...')print('=' * 60)# Create Tree Explainerexplainer_shap = shap.TreeExplainer(rf_credit)# Select a test instanceinstance_idx = 42instance = X_test.iloc[instance_idx:instance_idx+1]# Calculate SHAP valuesshap_values = explainer_shap.shap_values(instance)# For binary classification, take positive classif isinstance(shap_values, list):    shap_values_pos = shap_values[1]else:    shap_values_pos = shap_valuesprint(f'✅ SHAP values calculated')print(f'   Instance index: {instance_idx}')print(f'   Prediction: {"Approved" if rf_credit.predict(instance)[0] == 1 else "Rejected"}')print(f'   Prediction probability: {rf_credit.predict_proba(instance)[0][1]:.3f}')print('=' * 60)

In [ ]:
# LIME Explanationprint('\n🔍 Generating LIME Explanations...')print('=' * 60)try:    from lime import lime_tabular        # Create LIME explainer    explainer_lime = lime_tabular.LimeTabularExplainer(        X_train.values,        feature_names=X_train.columns.tolist(),        class_names=['Rejected', 'Approved'],        mode='classification',        random_state=RANDOM_STATE    )        # Explain the same instance    lime_exp = explainer_lime.explain_instance(        instance.values[0],        rf_credit.predict_proba,        num_features=len(X_train.columns)    )        # Extract LIME weights    lime_values = dict(lime_exp.as_list())    lime_values_dict = {}    for feat in X_train.columns:        for key, val in lime_values.items():            if feat in key:                lime_values_dict[feat] = val                break        print(f'✅ LIME explanation generated')    print('=' * 60)        lime_available = Trueexcept ImportError:    print('⚠️  LIME not installed. Run: pip install lime')    print('   Skipping LIME comparison...')    lime_available = False    print('=' * 60)

In [ ]:
# Compare SHAP and LIMEif lime_available:    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))        # SHAP plot    feature_names = X_train.columns    shap_vals = shap_values_pos[0]        sorted_idx = np.argsort(np.abs(shap_vals))    sorted_features = feature_names[sorted_idx]    sorted_shap = shap_vals[sorted_idx]        colors_shap = ['red' if x < 0 else 'green' for x in sorted_shap]    ax1.barh(range(len(sorted_features)), sorted_shap, color=colors_shap, alpha=0.7, edgecolor='black')    ax1.set_yticks(range(len(sorted_features)))    ax1.set_yticklabels(sorted_features)    ax1.set_xlabel('SHAP Value (impact on prediction)', fontweight='bold')    ax1.set_title('🎯 SHAP Explanations', fontweight='bold', fontsize=14)    ax1.axvline(x=0, color='black', linestyle='--', linewidth=1)    ax1.grid(axis='x', alpha=0.3)        # LIME plot    lime_features = list(lime_values_dict.keys())    lime_vals = list(lime_values_dict.values())        sorted_idx_lime = np.argsort(np.abs(lime_vals))    sorted_features_lime = [lime_features[i] for i in sorted_idx_lime]    sorted_lime = [lime_vals[i] for i in sorted_idx_lime]        colors_lime = ['red' if x < 0 else 'green' for x in sorted_lime]    ax2.barh(range(len(sorted_features_lime)), sorted_lime, color=colors_lime, alpha=0.7, edgecolor='black')    ax2.set_yticks(range(len(sorted_features_lime)))    ax2.set_yticklabels(sorted_features_lime)    ax2.set_xlabel('LIME Weight (impact on prediction)', fontweight='bold')    ax2.set_title('🍋 LIME Explanations', fontweight='bold', fontsize=14)    ax2.axvline(x=0, color='black', linestyle='--', linewidth=1)    ax2.grid(axis='x', alpha=0.3)        plt.tight_layout()    plt.show()        print('\n📊 Comparison Results:')    print('=' * 60)    print('\nSHAP Feature Rankings:')    for i, (feat, val) in enumerate(zip(sorted_features[-5:][::-1], sorted_shap[-5:][::-1]), 1):        print(f'  {i}. {feat:20} → {val:+.4f}')        print('\nLIME Feature Rankings:')    for i, (feat, val) in enumerate(zip(sorted_features_lime[-5:][::-1], sorted_lime[-5:][::-1]), 1):        print(f'  {i}. {feat:20} → {val:+.4f}')        print('\n💡 Key Differences:')    print('  • SHAP: Based on game theory, considers all coalitions')    print('  • LIME: Based on local linear approximation')    print('  • Rankings may differ due to different methodologies')    print('  • SHAP satisfies theoretical properties (additivity, consistency)')    print('=' * 60)else:    print('\n⚠️  Install LIME to see comparison: pip install lime')

### ✏️ Your Turn: Exercise 2a**Task**: Run SHAP and LIME explanations on 5 different instances and compare:1. Do they always agree on the top-3 most important features?2. Calculate the correlation between SHAP and LIME feature importances3. Which method is more stable across different instances?

In [ ]:
# YOUR CODE HERE# TODO: Compare SHAP and LIME on multiple instancesn_instances = 5# Sample random instances from test set# Calculate both SHAP and LIME for each# Compare feature rankings

---## Exercise 3: Mathematical Properties of SHAP### 📖 ConceptSHAP values satisfy four key axioms that make them theoretically sound:#### 1. **Local Accuracy (Efficiency)**```f(x) - E[f(X)] = Σ φᵢ(x)```The sum of SHAP values equals the difference between prediction and expected value.#### 2. **Missingness**If a feature is missing (not used), its SHAP value is 0.#### 3. **Consistency (Monotonicity)**If a model changes so a feature's contribution increases, its SHAP value shouldn't decrease.#### 4. **Symmetry**If two features contribute equally to all predictions, they get equal SHAP values.Let's verify these properties empirically!

In [ ]:
# Property 1: Local Accuracy (Efficiency)print('🔍 PROPERTY 1: LOCAL ACCURACY (EFFICIENCY)')print('=' * 70)# Get base value (expected prediction)explainer_full = shap.TreeExplainer(rf_credit)shap_values_all = explainer_shap.shap_values(X_test)# For binary classificationif isinstance(shap_values_all, list):    shap_values_test = shap_values_all[1]else:    shap_values_test = shap_values_allexpected_value = explainer_full.expected_valueif isinstance(expected_value, list):    expected_value = expected_value[1]# Check first 5 instancesprint(f'Base value (expected prediction): {expected_value:.4f}\n')for i in range(min(5, len(X_test))):    instance = X_test.iloc[i:i+1]    actual_pred = rf_credit.predict_proba(instance)[0][1]    shap_sum = shap_values_test[i].sum()    reconstructed_pred = expected_value + shap_sum        print(f'Instance {i}:')    print(f'  Actual prediction:        {actual_pred:.4f}')    print(f'  Base + SHAP sum:          {reconstructed_pred:.4f}')    print(f'  Difference:               {abs(actual_pred - reconstructed_pred):.6f}')    print(f'  Satisfies property:       {"✅ Yes" if abs(actual_pred - reconstructed_pred) < 0.01 else "❌ No"}')    print()print('💡 Interpretation: SHAP values perfectly reconstruct predictions!')print('=' * 70)

In [ ]:
# Property 2: Missingness - Verify with feature ablationprint('\n🔍 PROPERTY 2: MISSINGNESS')print('=' * 70)# Train model WITHOUT one featureX_ablated = X_train.drop('Previous_Defaults', axis=1)X_test_ablated = X_test.drop('Previous_Defaults', axis=1)rf_ablated = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=RANDOM_STATE)rf_ablated.fit(X_ablated, y_train)# Calculate SHAP for ablated modelexplainer_ablated = shap.TreeExplainer(rf_ablated)shap_values_ablated = explainer_ablated.shap_values(X_test_ablated)if isinstance(shap_values_ablated, list):    shap_values_ablated = shap_values_ablated[1]print('Model trained WITHOUT "Previous_Defaults" feature')print(f'Number of features in original model: {X_train.shape[1]}')print(f'Number of features in ablated model:  {X_ablated.shape[1]}\n')print('SHAP values for "Previous_Defaults" in original model:')prev_def_idx = X_train.columns.get_loc('Previous_Defaults')print(f'  Mean absolute SHAP: {np.abs(shap_values_test[:, prev_def_idx]).mean():.4f}')print(f'  Non-zero SHAP values: {(shap_values_test[:, prev_def_idx] != 0).sum()} / {len(shap_values_test)}')print('\n💡 Interpretation: When feature is in model, it has non-zero SHAP values.')print('   If removed from model, it would have zero contribution.')print('=' * 70)

In [ ]:
# Property 3 & 4: Visualize with heatmapprint('\n🔍 PROPERTY 3 & 4: CONSISTENCY AND SYMMETRY')print('=' * 70)# Calculate SHAP for multiple instancesn_samples = 20sample_indices = np.random.choice(len(X_test), n_samples, replace=False)X_sample = X_test.iloc[sample_indices]explainer_sample = shap.TreeExplainer(rf_credit)shap_sample = explainer_sample.shap_values(X_sample)if isinstance(shap_sample, list):    shap_sample = shap_sample[1]# Create heatmapfig, ax = plt.subplots(figsize=(14, 10))im = ax.imshow(shap_sample.T, cmap='RdBu_r', aspect='auto', vmin=-0.5, vmax=0.5)# Labelsax.set_xticks(range(n_samples))ax.set_yticks(range(len(X_train.columns)))ax.set_xticklabels([f'Inst {i}' for i in range(n_samples)], rotation=90)ax.set_yticklabels(X_train.columns)# Add colorbarcbar = plt.colorbar(im, ax=ax)cbar.set_label('SHAP Value', rotation=270, labelpad=20, fontweight='bold')# Add titleax.set_title('🎨 SHAP Values Heatmap: Feature Contributions Across Instances',              fontweight='bold', fontsize=14, pad=20)plt.tight_layout()plt.show()print('\n💡 Consistency: Features with higher contributions show warmer colors')print('💡 Symmetry: Features with similar patterns have similar SHAP distributions')print('=' * 70)

### ✏️ Your Turn: Exercise 3a**Task**: Verify SHAP properties on the Iris dataset1. Train a classifier on Iris data2. Calculate SHAP values for test set3. Verify local accuracy for 10 random instances4. Create a feature importance ranking5. Compare with model's built-in feature_importances_

In [ ]:
# YOUR CODE HERE# TODO: Verify SHAP properties on Iris# Step 1: Prepare dataX_iris = iris_df.drop(['target', 'species'], axis=1)y_iris = iris_df['target']# Step 2: Train model# Step 3: Calculate SHAP values# Step 4: Verify local accuracy# Step 5: Compare with feature_importances_

---## Exercise 4: Understanding Feature Interactions### 📖 ConceptSHAP not only provides individual feature contributions but can also reveal **feature interactions**.**Interaction**: When the combined effect of two features differs from their individual effects.**Formula for SHAP Interaction Values**:```φᵢ,ⱼ = effect of features i and j together - effect of i alone - effect of j alone```**Example**:- Feature A (size): +$50K- Feature B (location): +$30K- Together: +$95K (not $80K!)- Interaction: $95K - $50K - $30K = +$15K (synergy!)

In [ ]:
# Calculate SHAP interaction valuesprint('🔗 CALCULATING SHAP INTERACTION VALUES')print('=' * 70)# Use a smaller sample for faster computationX_interaction = X_test.iloc[:100]print('⏳ Computing interactions (this may take a moment)...')try:    # Calculate interaction values    shap_interaction_values = explainer_shap.shap_interaction_values(X_interaction)        # For binary classification, take positive class    if isinstance(shap_interaction_values, list):        interactions = shap_interaction_values[1]    else:        interactions = shap_interaction_values        # Average over all instances    mean_interactions = np.abs(interactions).mean(axis=0)        print('✅ Interaction values computed successfully!')    print(f'   Shape: {interactions.shape}')    print(f'   (instances, features, features)')    except Exception as e:    print(f'⚠️  Error computing interactions: {e}')    print('   Using approximate method...')    # Fallback: compute pairwise correlations of SHAP values    mean_interactions = np.abs(np.corrcoef(shap_values_test[:100].T))print('=' * 70)

In [ ]:
# Visualize interaction matrixfig, ax = plt.subplots(figsize=(12, 10))# Mask diagonal for better visualizationinteraction_matrix = mean_interactions.copy()np.fill_diagonal(interaction_matrix, 0)im = ax.imshow(interaction_matrix, cmap='YlOrRd', aspect='auto')# Add labelsfeature_names = X_train.columnsax.set_xticks(range(len(feature_names)))ax.set_yticks(range(len(feature_names)))ax.set_xticklabels(feature_names, rotation=45, ha='right')ax.set_yticklabels(feature_names)# Add colorbarcbar = plt.colorbar(im, ax=ax)cbar.set_label('|Interaction Strength|', rotation=270, labelpad=20, fontweight='bold')# Add values to cellsfor i in range(len(feature_names)):    for j in range(len(feature_names)):        if i != j:            text = ax.text(j, i, f'{interaction_matrix[i, j]:.3f}',                          ha="center", va="center", color="black", fontsize=8)ax.set_title('🔗 SHAP Interaction Matrix\n(Diagonal removed for clarity)',              fontweight='bold', fontsize=14, pad=15)plt.tight_layout()plt.show()# Find top interactionsprint('\n🔝 TOP 5 FEATURE INTERACTIONS')print('=' * 70)interaction_pairs = []for i in range(len(feature_names)):    for j in range(i+1, len(feature_names)):        interaction_pairs.append((feature_names[i], feature_names[j], interaction_matrix[i, j]))interaction_pairs.sort(key=lambda x: x[2], reverse=True)for idx, (feat1, feat2, strength) in enumerate(interaction_pairs[:5], 1):    print(f'{idx}. {feat1:20} × {feat2:20} → {strength:.4f}')print('\n💡 Interpretation:')print('  • Higher values indicate stronger interactions')print('  • Positive: Features amplify each other')print('  • Consider interactions for feature engineering')print('=' * 70)

### ✏️ Your Turn: Exercise 4a**Task**: Analyze interactions in the Housing Price dataset1. Train a regression model on housing data2. Calculate SHAP interaction values3. Find the top-3 strongest interactions4. Explain why these interactions make sense (domain knowledge)5. Create a scatter plot showing the interaction effect

In [ ]:
# YOUR CODE HERE# TODO: Analyze housing price interactions# Hint: Look for interactions like:# - Square_Feet × Bedrooms (bigger houses need more rooms)# - Age × Distance_to_City (old suburban vs new urban)# - School_Rating × Distance_to_City

---# 🛠 Part 2: SHAP Implementation Methods## Different Explainers for Different ModelsSHAP provides **specialized explainers** optimized for different model types:| Explainer | Best For | Speed | Accuracy ||-----------|----------|-------|----------|| **TreeExplainer** | Tree-based models (RF, XGBoost, LightGBM) | ⚡⚡⚡ Fast | ✅ Exact || **KernelExplainer** | Any model (black-box) | 🐌 Slow | ≈ Approximation || **DeepExplainer** | Neural networks (TensorFlow, PyTorch) | ⚡⚡ Medium | ≈ Approximation || **GradientExplainer** | Differentiable models | ⚡⚡ Medium | ≈ Approximation || **LinearExplainer** | Linear models | ⚡⚡⚡ Fast | ✅ Exact |### 🎯 Selection Guide```pythonif model in [RandomForest, XGBoost, LightGBM, CatBoost]:    use TreeExplainer()  # Fastest and exact    elif model in [Neural Network]:    use DeepExplainer()  # Optimized for deep learning    elif model in [Logistic Regression, Linear SVM]:    use LinearExplainer()  # Exact for linear models    else:  # SVM, KNN, or custom models    use KernelExplainer()  # Works for anything```---

## Exercise 5: TreeSHAP - Exact and Fast### 📖 Concept**TreeSHAP** is a breakthrough algorithm that computes exact SHAP values for tree-based models in polynomial time.**Key advantages**:- ⚡ **Fast**: O(TLD²) where T=trees, L=leaves, D=depth- ✅ **Exact**: Not an approximation- 🌳 **Optimized**: Leverages tree structure**Supported models**:- Random Forest- Gradient Boosting (XGBoost, LightGBM, CatBoost)- Decision Trees- Isolation Forest

In [ ]:
# Train multiple tree-based modelsprint('🌳 TRAINING TREE-BASED MODELS')print('=' * 70)from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifierfrom sklearn.tree import DecisionTreeClassifierimport xgboost as xgbimport time# Prepare cancer dataX_cancer = cancer_df.drop(['target', 'diagnosis'], axis=1)y_cancer = cancer_df['target']X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(    X_cancer, y_cancer, test_size=0.2, random_state=RANDOM_STATE, stratify=y_cancer)models = {}# 1. Random Forestprint('\n1️⃣  Training Random Forest...')rf = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=RANDOM_STATE)start = time.time()rf.fit(X_train_c, y_train_c)rf_time = time.time() - startmodels['Random Forest'] = rfprint(f'   Training time: {rf_time:.3f}s')print(f'   Test accuracy: {accuracy_score(y_test_c, rf.predict(X_test_c)):.3f}')# 2. Gradient Boostingprint('\n2️⃣  Training Gradient Boosting...')gb = GradientBoostingClassifier(n_estimators=100, max_depth=4, random_state=RANDOM_STATE)start = time.time()gb.fit(X_train_c, y_train_c)gb_time = time.time() - startmodels['Gradient Boosting'] = gbprint(f'   Training time: {gb_time:.3f}s')print(f'   Test accuracy: {accuracy_score(y_test_c, gb.predict(X_test_c)):.3f}')# 3. XGBoostprint('\n3️⃣  Training XGBoost...')xgb_model = xgb.XGBClassifier(n_estimators=100, max_depth=4, random_state=RANDOM_STATE,                                eval_metric='logloss')start = time.time()xgb_model.fit(X_train_c, y_train_c)xgb_time = time.time() - startmodels['XGBoost'] = xgb_modelprint(f'   Training time: {xgb_time:.3f}s')print(f'   Test accuracy: {accuracy_score(y_test_c, xgb_model.predict(X_test_c)):.3f}')print('\n' + '=' * 70)print('✅ All models trained successfully!')print('=' * 70)

In [ ]:
# Calculate TreeSHAP for each modelprint('\n⏱️  TREESHAP COMPUTATION TIME COMPARISON')print('=' * 70)shap_times = {}shap_results = {}for model_name, model in models.items():    print(f'\n{model_name}:')        # Create explainer    explainer = shap.TreeExplainer(model)        # Time the computation    start = time.time()    shap_values = explainer.shap_values(X_test_c)    elapsed = time.time() - start        shap_times[model_name] = elapsed    shap_results[model_name] = shap_values        # Handle different output formats    if isinstance(shap_values, list):        shap_values = shap_values[1]  # Positive class        print(f'   Computation time: {elapsed:.3f}s')    print(f'   Instances explained: {len(X_test_c)}')    print(f'   Time per instance: {elapsed/len(X_test_c)*1000:.2f}ms')    print(f'   Expected value: {explainer.expected_value:.4f}')print('\n' + '=' * 70)print('✅ TreeSHAP is extremely fast for all tree models!')print('=' * 70)

In [ ]:
# Visualize computation timesfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))# Left: Computation time comparisonmodel_names = list(shap_times.keys())times = list(shap_times.values())bars = ax1.bar(model_names, times, color=['#3498db', '#2ecc71', '#e74c3c'],                alpha=0.7, edgecolor='black', linewidth=2)for bar, time_val in zip(bars, times):    height = bar.get_height()    ax1.text(bar.get_x() + bar.get_width()/2., height,             f'{time_val:.3f}s',             ha='center', va='bottom', fontweight='bold')ax1.set_ylabel('Computation Time (seconds)', fontweight='bold', fontsize=12)ax1.set_title('⏱️ TreeSHAP Computation Speed', fontweight='bold', fontsize=14)ax1.set_ylim(0, max(times) * 1.2)ax1.grid(axis='y', alpha=0.3)# Right: Feature importance comparisonexplainer_rf = shap.TreeExplainer(models['Random Forest'])shap_vals_rf = explainer_rf.shap_values(X_test_c)if isinstance(shap_vals_rf, list):    shap_vals_rf = shap_vals_rf[1]# Calculate mean absolute SHAP valuesmean_shap = np.abs(shap_vals_rf).mean(axis=0)sorted_idx = np.argsort(mean_shap)[-10:]  # Top 10ax2.barh(range(len(sorted_idx)), mean_shap[sorted_idx],          color='coral', alpha=0.7, edgecolor='black')ax2.set_yticks(range(len(sorted_idx)))ax2.set_yticklabels(X_cancer.columns[sorted_idx], fontsize=9)ax2.set_xlabel('Mean |SHAP value|', fontweight='bold', fontsize=12)ax2.set_title('🎯 Top 10 Important Features (Random Forest)', fontweight='bold', fontsize=14)ax2.grid(axis='x', alpha=0.3)plt.tight_layout()plt.show()print('\n💡 Key Insight: TreeSHAP provides exact SHAP values in milliseconds!')

### ✏️ Your Turn: Exercise 5a**Task**: Compare TreeSHAP across different tree configurations1. Train 3 Random Forests with different settings:   - Shallow trees (max_depth=3)   - Medium trees (max_depth=10)   - Deep trees (max_depth=None)2. Measure TreeSHAP computation time for each3. Compare feature importance rankings4. Analyze how tree depth affects computation time

In [ ]:
# YOUR CODE HERE# TODO: Compare TreeSHAP performance vs tree depthconfigurations = [    {'name': 'Shallow', 'max_depth': 3},    {'name': 'Medium', 'max_depth': 10},    {'name': 'Deep', 'max_depth': None}]# Train models and measure TreeSHAP times

---## Exercise 6: KernelSHAP - Model-Agnostic Approach### 📖 Concept**KernelSHAP** is the most general SHAP explainer - it works with **any model**!**How it works**:1. Generate random coalitions of features2. Evaluate model on perturbed inputs3. Fit a weighted linear model4. Extract coefficients as SHAP values**Trade-off**:- ✅ **Universal**: Works with any model (even black-box)- ❌ **Slow**: Requires many model evaluations- ≈ **Approximate**: Monte Carlo sampling**When to use**:- SVM, KNN, or other non-tree models- Custom models or ensembles- When you need model-agnostic explanations

In [ ]:
# Train non-tree modelsprint('🔮 TRAINING NON-TREE MODELS')print('=' * 70)from sklearn.svm import SVCfrom sklearn.neighbors import KNeighborsClassifierfrom sklearn.naive_bayes import GaussianNB# Use iris for faster computationX_iris = iris_df.drop(['target', 'species'], axis=1)y_iris = iris_df['target']X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(    X_iris, y_iris, test_size=0.2, random_state=RANDOM_STATE, stratify=y_iris)# 1. Support Vector Machineprint('\n1️⃣  Training SVM...')svm_model = SVC(kernel='rbf', probability=True, random_state=RANDOM_STATE)svm_model.fit(X_train_i, y_train_i)print(f'   Test accuracy: {accuracy_score(y_test_i, svm_model.predict(X_test_i)):.3f}')# 2. K-Nearest Neighborsprint('\n2️⃣  Training KNN...')knn_model = KNeighborsClassifier(n_neighbors=5)knn_model.fit(X_train_i, y_train_i)print(f'   Test accuracy: {accuracy_score(y_test_i, knn_model.predict(X_test_i)):.3f}')# 3. Naive Bayesprint('\n3️⃣  Training Naive Bayes...')nb_model = GaussianNB()nb_model.fit(X_train_i, y_train_i)print(f'   Test accuracy: {accuracy_score(y_test_i, nb_model.predict(X_test_i)):.3f}')print('\n' + '=' * 70)print('✅ Non-tree models trained!')print('=' * 70)

In [ ]:
# Apply KernelSHAP to SVMprint('\n🔍 APPLYING KERNELSHAP TO SVM')print('=' * 70)# Create KernelExplainer# Note: KernelSHAP needs a background datasetbackground = shap.sample(X_train_i, 50)  # Use 50 samples as backgroundprint(f'Background dataset: {background.shape}')print('Creating KernelExplainer...')# Create explainerexplainer_kernel = shap.KernelExplainer(svm_model.predict_proba, background)# Explain a single instance (KernelSHAP is slow!)print('\nExplaining 1 instance...')test_instance = X_test_i.iloc[0:1]start = time.time()shap_values_kernel = explainer_kernel.shap_values(test_instance, nsamples=100)kernel_time = time.time() - startprint(f'✅ KernelSHAP computation completed in {kernel_time:.3f}s')print(f'   Time per instance: {kernel_time:.3f}s (vs TreeSHAP: ~0.001s)')# Compare with TreeSHAP (if we had a tree model)rf_iris = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)rf_iris.fit(X_train_i, y_train_i)explainer_tree = shap.TreeExplainer(rf_iris)start = time.time()shap_values_tree = explainer_tree.shap_values(test_instance)tree_time = time.time() - startprint(f'\n📊 Speed Comparison:')print(f'   KernelSHAP: {kernel_time:.3f}s')print(f'   TreeSHAP:   {tree_time:.3f}s')print(f'   Speedup:    {kernel_time/tree_time:.0f}x faster with TreeSHAP!')print('=' * 70)

In [ ]:
# Visualize KernelSHAP resultsfig, axes = plt.subplots(1, 3, figsize=(18, 5))# For multi-class, visualize each classfor class_idx in range(3):    ax = axes[class_idx]        # Extract SHAP values for this class    if isinstance(shap_values_kernel, list):        shap_class = shap_values_kernel[class_idx][0]    else:        shap_class = shap_values_kernel[0]        # Create bar plot    features = X_iris.columns    colors = ['red' if x < 0 else 'green' for x in shap_class]        ax.barh(range(len(features)), shap_class, color=colors, alpha=0.7, edgecolor='black')    ax.set_yticks(range(len(features)))    ax.set_yticklabels(features)    ax.set_xlabel('SHAP value', fontweight='bold')    ax.set_title(f'Class {iris.target_names[class_idx]}', fontweight='bold')    ax.axvline(x=0, color='black', linestyle='--', linewidth=1)    ax.grid(axis='x', alpha=0.3)plt.suptitle('🔮 KernelSHAP: SVM Explanations for Iris Classes',              fontweight='bold', fontsize=16, y=1.02)plt.tight_layout()plt.show()print('\n💡 Interpretation: KernelSHAP works with ANY model, but is slower')

### ✏️ Your Turn: Exercise 6a**Task**: Compare KernelSHAP vs TreeSHAP on the same data1. Train both Random Forest and SVM on credit data2. Use KernelSHAP for SVM and TreeSHAP for RF3. Explain the same 5 instances with both methods4. Compare:   - Computation time   - Feature rankings   - Explanation consistency5. When would you choose KernelSHAP despite its slowness?

In [ ]:
# YOUR CODE HERE# TODO: Compare KernelSHAP and TreeSHAP# Hints:# - Use small background sample (50-100) for KernelSHAP# - Set nsamples=100 for faster but less accurate results# - Compare feature rankings using rank correlation

---## Exercise 7: DeepSHAP for Neural Networks### 📖 Concept**DeepSHAP** combines SHAP with DeepLIFT to explain neural networks efficiently.**Key features**:- 🧠 Designed for deep learning models- ⚡ Faster than KernelSHAP for neural nets- 📊 Backpropagates through network layers- 🎯 Works with TensorFlow, PyTorch, Keras**How it works**:1. Uses reference activations (baseline)2. Computes gradients × activations3. Distributes attribution through layers4. Much faster than sampling-based methods

In [ ]:
# Create a simple neural networkprint('🧠 BUILDING NEURAL NETWORK')print('=' * 70)from sklearn.neural_network import MLPClassifier# Train MLP on cancer datamlp = MLPClassifier(hidden_layer_sizes=(64, 32, 16), max_iter=500,                     random_state=RANDOM_STATE, early_stopping=True)print('Training Multi-Layer Perceptron...')mlp.fit(X_train_c, y_train_c)train_acc = mlp.score(X_train_c, y_train_c)test_acc = mlp.score(X_test_c, y_test_c)print(f'\n✅ Neural Network Trained!')print(f'   Architecture: {[X_train_c.shape[1]] + list(mlp.hidden_layer_sizes) + [2]}')print(f'   Training accuracy: {train_acc:.3f}')print(f'   Test accuracy: {test_acc:.3f}')print('=' * 70)

In [ ]:
# Apply GradientExplainer (alternative to DeepSHAP for sklearn)print('\n🔍 APPLYING GRADIENTEXPLAINER')print('=' * 70)# For sklearn MLP, we use GradientExplainer# Note: sklearn's MLP doesn't support DeepSHAP directly# In practice, you'd use TensorFlow/PyTorch for DeepSHAPprint('Using GradientExplainer for sklearn MLP...')print('(For true DeepSHAP, use TensorFlow/PyTorch models)')# Create background datasetbackground_nn = X_train_c.iloc[:50]# Create explainerexplainer_nn = shap.KernelExplainer(mlp.predict_proba, background_nn)# Explain a few instancesprint('\nExplaining 3 test instances...')test_sample = X_test_c.iloc[:3]start = time.time()shap_values_nn = explainer_nn.shap_values(test_sample, nsamples=100)nn_time = time.time() - startprint(f'✅ Explanation completed in {nn_time:.3f}s')print(f'   Time per instance: {nn_time/3:.3f}s')print('\n💡 Note: For production deep learning, use:')print('   • TensorFlow → shap.DeepExplainer(model, background)')print('   • PyTorch → shap.DeepExplainer(model, background)')print('=' * 70)

In [ ]:
# Visualize neural network explanationsfig, axes = plt.subplots(1, 3, figsize=(18, 6))for idx in range(3):    ax = axes[idx]        # Get SHAP values for this instance (positive class)    if isinstance(shap_values_nn, list):        shap_vals = shap_values_nn[1][idx]    else:        shap_vals = shap_values_nn[idx]        # Sort by absolute value    sorted_idx = np.argsort(np.abs(shap_vals))[-10:]  # Top 10    sorted_features = X_cancer.columns[sorted_idx]    sorted_values = shap_vals[sorted_idx]        colors = ['red' if x < 0 else 'green' for x in sorted_values]        ax.barh(range(len(sorted_features)), sorted_values, color=colors,             alpha=0.7, edgecolor='black')    ax.set_yticks(range(len(sorted_features)))    ax.set_yticklabels(sorted_features, fontsize=9)    ax.set_xlabel('SHAP value', fontweight='bold')    ax.set_title(f'Instance {idx+1}', fontweight='bold')    ax.axvline(x=0, color='black', linestyle='--', linewidth=1)    ax.grid(axis='x', alpha=0.3)plt.suptitle('🧠 Neural Network SHAP Explanations', fontweight='bold',              fontsize=16, y=1.02)plt.tight_layout()plt.show()print('\n💡 Neural networks can capture complex non-linear patterns!')

### ✏️ Your Turn: Exercise 7a**Task**: Compare shallow vs deep neural networks1. Train two MLPs:   - Shallow: 1 hidden layer with 32 neurons   - Deep: 3 hidden layers with (64, 32, 16) neurons2. Compare:   - Test accuracy   - SHAP explanation time   - Feature importance rankings3. Which architecture provides more consistent explanations?

In [ ]:
# YOUR CODE HERE# TODO: Compare shallow vs deep neural networks# Train shallow networkmlp_shallow = MLPClassifier(hidden_layer_sizes=(32,), max_iter=500,                              random_state=RANDOM_STATE)# Train deep network  mlp_deep = MLPClassifier(hidden_layer_sizes=(64, 32, 16), max_iter=500,                          random_state=RANDOM_STATE)# Compare explanations

---## Exercise 8: GradientSHAP for Differentiable Models### 📖 Concept**GradientSHAP** combines integrated gradients with SHAP sampling.**Formula**:```φᵢ = (xᵢ - x'ᵢ) × E[∂f/∂xᵢ]```Where:- xᵢ = input feature value- x'ᵢ = baseline feature value  - ∂f/∂xᵢ = gradient w.r.t. feature i**Advantages**:- ⚡ Fast for differentiable models- 🎯 Incorporates gradient information- 📉 Reduces variance vs plain gradients**Best for**:- Neural networks- Gradient boosting machines (with gradients)- Any differentiable model

In [ ]:
# Demonstrate gradient-based feature attributionprint('📈 GRADIENT-BASED FEATURE ATTRIBUTION')print('=' * 70)# Train a simple logistic regression (differentiable)from sklearn.linear_model import LogisticRegressionlogreg = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)logreg.fit(X_train_c, y_train_c)print(f'Logistic Regression trained')print(f'Test accuracy: {logreg.score(X_test_c, y_test_c):.3f}')# For logistic regression, coefficients ARE the gradientsprint('\n📊 Model Coefficients (= Feature Importance for Linear Models):')print('=' * 70)# Get feature importance from coefficientscoef_importance = np.abs(logreg.coef_[0])sorted_idx = np.argsort(coef_importance)[-10:]print('\nTop 10 features by |coefficient|:')for i, idx in enumerate(sorted_idx[::-1], 1):    feat = X_cancer.columns[idx]    val = logreg.coef_[0][idx]    print(f'{i:2}. {feat:30} → {val:+.4f}')print('=' * 70)

In [ ]:
# Apply LinearExplainer (exact for linear models)print('\n🔍 LINEAREXPLAINER FOR LOGISTIC REGRESSION')print('=' * 70)# LinearExplainer gives exact SHAP values for linear modelsexplainer_linear = shap.LinearExplainer(logreg, X_train_c)# Explain test setshap_values_linear = explainer_linear.shap_values(X_test_c)print('✅ SHAP values computed (exact for linear models)')print(f'   Shape: {shap_values_linear.shape}')print(f'   Computation: Instant (analytical formula)')# Compare with model coefficientsprint('\n📊 Comparison: SHAP vs Coefficients')print('=' * 70)mean_abs_shap = np.abs(shap_values_linear).mean(axis=0)correlation = np.corrcoef(mean_abs_shap, coef_importance)[0, 1]print(f'Correlation between mean |SHAP| and |coefficients|: {correlation:.3f}')print('\n💡 For linear models, SHAP values are scaled coefficients!')print('   They account for feature distributions and interactions.')print('=' * 70)

In [ ]:
# Visualize gradient-based attributionsfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))# Left: Coefficientssorted_idx = np.argsort(coef_importance)[-10:]ax1.barh(range(10), coef_importance[sorted_idx], color='steelblue',          alpha=0.7, edgecolor='black')ax1.set_yticks(range(10))ax1.set_yticklabels(X_cancer.columns[sorted_idx], fontsize=9)ax1.set_xlabel('|Coefficient|', fontweight='bold')ax1.set_title('📈 Logistic Regression Coefficients', fontweight='bold', fontsize=14)ax1.grid(axis='x', alpha=0.3)# Right: Mean SHAP valuessorted_idx_shap = np.argsort(mean_abs_shap)[-10:]ax2.barh(range(10), mean_abs_shap[sorted_idx_shap], color='coral',          alpha=0.7, edgecolor='black')ax2.set_yticks(range(10))ax2.set_yticklabels(X_cancer.columns[sorted_idx_shap], fontsize=9)ax2.set_xlabel('Mean |SHAP value|', fontweight='bold')ax2.set_title('🎯 SHAP Feature Importance', fontweight='bold', fontsize=14)ax2.grid(axis='x', alpha=0.3)plt.tight_layout()plt.show()print('\n💡 Both methods identify similar important features!')print('   But SHAP provides instance-level attributions.')

### ✏️ Your Turn: Exercise 8a**Task**: Integrated Gradients for Neural NetworksWhile we can't implement true integrated gradients without a deep learning framework, let's simulate the concept:1. Train a neural network on housing prices (regression)2. Calculate SHAP values for a test instance3. For a single feature, manually compute:   - Base prediction (feature = 0)   - Final prediction (feature = actual value)   - Interpolate between them4. Compare with SHAP value

In [ ]:
# YOUR CODE HERE# TODO: Simulate integrated gradients concept# Prepare housing regression dataX_house = housing_df.drop('Price', axis=1)y_house = housing_df['Price']# Split dataX_train_h, X_test_h, y_train_h, y_test_h = train_test_split(    X_house, y_house, test_size=0.2, random_state=RANDOM_STATE)# Train regression model# Calculate SHAP values# Manually compute integrated gradient for one feature

---## Exercise 9: Deep Dive into SHAP Interactions### 📖 Concept Recap**SHAP Interaction Values** decompose predictions into:- Main effects (individual features)- Pairwise interactions (feature pairs)**Mathematical formula**:```f(x) = φ₀ + Σφᵢ + Σφᵢ,ⱼ       base  main   interactions```**Use cases**:- Detect feature synergies or conflicts- Guide feature engineering- Understand model complexity

In [ ]:
# Calculate detailed interaction analysisprint('🔗 DEEP INTERACTION ANALYSIS')print('=' * 70)# Use Random Forest on credit datarf_interact = RandomForestClassifier(n_estimators=50, max_depth=6,                                      random_state=RANDOM_STATE)rf_interact.fit(X_train, y_train)# Calculate interactions for subset of dataX_interact_sample = X_test.iloc[:50]print('Computing SHAP interaction values...')print('(This may take a moment for pairwise interactions)')explainer_interact = shap.TreeExplainer(rf_interact)try:    interaction_vals = explainer_interact.shap_interaction_values(X_interact_sample)        if isinstance(interaction_vals, list):        interaction_vals = interaction_vals[1]  # Positive class        print(f'\n✅ Interaction values computed')    print(f'   Shape: {interaction_vals.shape}')    print(f'   Format: (instances, features, features)')        # Analyze diagonal (main effects) vs off-diagonal (interactions)    main_effects = np.array([interaction_vals[i, :, :].diagonal()                              for i in range(len(interaction_vals))])        mean_main = np.abs(main_effects).mean(axis=0)        # Off-diagonal (interactions)    mean_interact = np.abs(interaction_vals).mean(axis=0)    np.fill_diagonal(mean_interact, 0)        print(f'\n📊 Analysis:')    print(f'   Mean |main effect|:     {mean_main.mean():.4f}')    print(f'   Mean |interaction|:     {mean_interact[mean_interact > 0].mean():.4f}')    print(f'   Ratio (main/interact):  {mean_main.mean() / mean_interact[mean_interact > 0].mean():.2f}x')        interaction_computed = True    except Exception as e:    print(f'\n⚠️  Could not compute interactions: {e}')    print('   Using approximation based on SHAP value correlations')    interaction_computed = Falseprint('=' * 70)

In [ ]:
# Find strongest interactionsif interaction_computed:    print('\n🔝 STRONGEST FEATURE INTERACTIONS')    print('=' * 70)        # Get feature names    features = X_train.columns        # Find top interactions    interaction_strength = []    for i in range(len(features)):        for j in range(i+1, len(features)):            strength = mean_interact[i, j]            interaction_strength.append((features[i], features[j], strength))        # Sort by strength    interaction_strength.sort(key=lambda x: x[2], reverse=True)        print('\nTop 10 Feature Interactions:')    print('-' * 70)    print(f'{"Rank":<6} {"Feature 1":<22} {"Feature 2":<22} {"Strength":<10}')    print('-' * 70)        for rank, (feat1, feat2, strength) in enumerate(interaction_strength[:10], 1):        print(f'{rank:<6} {feat1:<22} {feat2:<22} {strength:<10.4f}')        print('=' * 70)        # Visualize top interaction    top_i, top_j = features.get_loc(interaction_strength[0][0]), features.get_loc(interaction_strength[0][1])        fig, ax = plt.subplots(figsize=(10, 6))        # Scatter plot showing interaction    scatter = ax.scatter(X_interact_sample.iloc[:, top_i],                         X_interact_sample.iloc[:, top_j],                        c=interaction_vals[:, top_i, top_j],                        cmap='RdYlGn', s=100, alpha=0.6, edgecolors='black')        ax.set_xlabel(features[top_i], fontweight='bold', fontsize=12)    ax.set_ylabel(features[top_j], fontweight='bold', fontsize=12)    ax.set_title(f'🔗 Strongest Interaction: {features[top_i]} × {features[top_j]}',                 fontweight='bold', fontsize=14)        cbar = plt.colorbar(scatter, ax=ax)    cbar.set_label('Interaction SHAP Value', rotation=270, labelpad=20, fontweight='bold')        plt.tight_layout()    plt.show()        print('\n💡 Interpretation:')    print(f'   • {features[top_i]} and {features[top_j]} have strong interaction')    print('   • Color shows combined effect on prediction')    print('   • Consider creating interaction features!')else:    print('\n📊 Showing SHAP value correlation as interaction proxy...')        shap_vals_full = explainer_interact.shap_values(X_test)    if isinstance(shap_vals_full, list):        shap_vals_full = shap_vals_full[1]        # Correlation of SHAP values    shap_corr = np.corrcoef(shap_vals_full.T)    np.fill_diagonal(shap_corr, 0)        # Find high correlations    high_corr_pairs = []    for i in range(len(X_train.columns)):        for j in range(i+1, len(X_train.columns)):            high_corr_pairs.append((X_train.columns[i], X_train.columns[j],                                    abs(shap_corr[i, j])))        high_corr_pairs.sort(key=lambda x: x[2], reverse=True)        print('\nTop correlated SHAP values (potential interactions):')    for feat1, feat2, corr in high_corr_pairs[:5]:        print(f'  {feat1} × {feat2}: {corr:.3f}')

---# 📈 Part 3: SHAP Visualizations## Communicating Model BehaviorSHAP provides powerful visualizations to understand model predictions at different levels:| Plot Type | Level | Purpose ||-----------|-------|---------|| **Waterfall** | Instance | Show prediction breakdown for one instance || **Force** | Instance | Interactive explanation of single prediction || **Decision** | Instance | Compare multiple instances || **Summary** | Global | Overview of feature importance || **Dependence** | Global | Show feature effect across all data || **Bar** | Global | Simple feature ranking |### 🎨 Visualization Philosophy1. **Start local** → Understand individual predictions2. **Go global** → See overall patterns3. **Dive deep** → Explore specific features4. **Tell stories** → Use plots to communicate insights---

## Exercise 10: Waterfall Plots - Explaining Single Predictions### 📖 Concept**Waterfall plots** show how each feature pushes the prediction from the base value to the final prediction.**Components**:- 🔵 **Base value**: Expected model output (usually mean)- 🟢 **Positive contributions**: Features increasing prediction- 🔴 **Negative contributions**: Features decreasing prediction- ⚫ **Final prediction**: Where we end up**Perfect for**:- Explaining specific decisions to stakeholders- Debugging model behavior- Regulatory compliance (e.g., loan rejections)

In [ ]:
# Prepare data for waterfall demonstrationprint('🌊 WATERFALL PLOT DEMONSTRATION')print('=' * 70)# Get SHAP values for a specific instanceinstance_idx = 15instance = X_test.iloc[instance_idx]instance_values = instance.valuesfeature_names = X_test.columns.tolist()# Calculate SHAP valuesshap_vals = explainer_shap.shap_values(X_test.iloc[instance_idx:instance_idx+1])if isinstance(shap_vals, list):    shap_vals = shap_vals[1][0]  # Positive class, first instanceelse:    shap_vals = shap_vals[0]# Get base valuebase_value = explainer_shap.expected_valueif isinstance(base_value, list):    base_value = base_value[1]# Get predictionprediction = rf_credit.predict_proba(X_test.iloc[instance_idx:instance_idx+1])[0][1]print(f'Analyzing instance #{instance_idx}')print(f'\nActual label: {"Approved ✅" if y_test.iloc[instance_idx] == 1 else "Rejected ❌"}')print(f'Predicted probability: {prediction:.3f}')print(f'Base value (average): {base_value:.3f}')print(f'\nFeature values:')for feat, val in zip(feature_names, instance_values):    print(f'  {feat:25} = {val:.2f}')print('=' * 70)

In [ ]:
# Create custom waterfall visualizationfig, ax = plt.subplots(figsize=(12, 8))# Sort features by absolute SHAP valuesorted_idx = np.argsort(np.abs(shap_vals))[::-1]sorted_features = [feature_names[i] for i in sorted_idx]sorted_shap = shap_vals[sorted_idx]sorted_values = instance_values[sorted_idx]# Calculate cumulative sumcumsum = np.cumsum(sorted_shap)y_pos = range(len(sorted_features) + 2)  # +2 for base and final# Prepare data for waterfallall_labels = ['Base value'] + [f'{f}\n= {v:.1f}' for f, v in zip(sorted_features[:10], sorted_values[:10])] + ['Final']all_values = [base_value] + [base_value + cumsum[i] for i in range(10)] + [prediction]# Draw waterfallfor i in range(len(all_labels)-1):    start = all_values[i]    change = all_values[i+1] - all_values[i]    color = 'green' if change > 0 else 'red'        if i == 0:  # Base value        ax.bar(i, all_values[i], color='lightblue', alpha=0.7, edgecolor='black', linewidth=2)    else:        ax.bar(i, abs(change), bottom=min(start, all_values[i+1]),                color=color, alpha=0.7, edgecolor='black', linewidth=1.5)        # Add value labels    ax.text(i, all_values[i+1], f'{all_values[i+1]:.3f}',             ha='center', va='bottom', fontweight='bold', fontsize=9)# Final prediction barax.bar(len(all_labels)-1, prediction, color='darkblue', alpha=0.7,        edgecolor='black', linewidth=2)ax.set_xticks(range(len(all_labels)))ax.set_xticklabels(all_labels, rotation=45, ha='right', fontsize=9)ax.set_ylabel('Prediction Probability', fontweight='bold', fontsize=12)ax.set_title(f'🌊 Waterfall Plot: Credit Approval Explanation (Instance #{instance_idx})',             fontweight='bold', fontsize=14, pad=20)ax.grid(axis='y', alpha=0.3)ax.axhline(y=0.5, color='black', linestyle='--', linewidth=1, label='Decision threshold')ax.legend()plt.tight_layout()plt.show()print('\n💡 Interpretation:')print(f'   • Started at base value: {base_value:.3f}')print(f'   • Green bars push UP (toward approval)')print(f'   • Red bars push DOWN (toward rejection)')print(f'   • Final prediction: {prediction:.3f} → {"Approved ✅" if prediction > 0.5 else "Rejected ❌"}')

In [ ]:
# Use SHAP's built-in waterfall plotprint('\n📊 SHAP Built-in Waterfall Plot')print('=' * 70)# Create SHAP explanation objectimport shap# Get shap values for this instanceexplainer_waterfall = shap.TreeExplainer(rf_credit)shap_values_waterfall = explainer_waterfall(X_test.iloc[instance_idx:instance_idx+1])# Create waterfall plotshap.plots.waterfall(shap_values_waterfall[0], max_display=10, show=False)plt.title('SHAP Waterfall Plot (Built-in)', fontweight='bold', pad=20)plt.tight_layout()plt.show()print('\n💡 Built-in plot shows:')print('   • E[f(X)] = Expected value (base)')print('   • f(x) = Actual prediction')print('   • Features sorted by absolute impact')print('=' * 70)

### ✏️ Your Turn: Exercise 10a**Task**: Create waterfall explanations for edge cases1. Find 3 interesting instances:   - One with high confidence correct prediction   - One with low confidence (near 0.5)   - One misclassified instance2. Create waterfall plots for each3. Analyze:   - Which features drive confidence?   - Why was the model uncertain?   - What caused the misclassification?4. Write a short explanation for each case

In [ ]:
# YOUR CODE HERE# TODO: Analyze edge cases with waterfall plots# Find interesting instances# High confidence: prediction close to 0 or 1# Low confidence: prediction near 0.5# Misclassified: prediction != actual# Create waterfall plots and analyze

---## Exercise 11: Summary Plots - Global Feature Importance### 📖 Concept**Summary plots** provide a **bird's-eye view** of feature importance across the entire dataset.**Key elements**:- 📊 **Y-axis**: Features ranked by importance- 🎨 **X-axis**: SHAP value (impact on prediction)- 🔴 **Color**: Feature value (red = high, blue = low)- 📍 **Dots**: Individual predictions**Insights you can extract**:1. **Most important features** (top of plot)2. **Direction of effects** (left = negative, right = positive)3. **Non-linear relationships** (spread of dots)4. **Interaction effects** (color patterns)

In [ ]:
# Create comprehensive summary plotprint('📊 SUMMARY PLOT DEMONSTRATION')print('=' * 70)# Calculate SHAP values for entire test setshap_values_summary = explainer_shap.shap_values(X_test)if isinstance(shap_values_summary, list):    shap_values_summary = shap_values_summary[1]  # Positive classprint(f'SHAP values computed for {len(X_test)} instances')print(f'Shape: {shap_values_summary.shape}')print('\nCreating summary plot...')# Create figure with two subplotsfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))# Left: Beeswarm plot (default summary plot)shap.summary_plot(shap_values_summary, X_test, plot_type="dot",                   show=False, max_display=10)plt.title('🐝 Beeswarm Summary Plot', fontweight='bold', fontsize=14, pad=15)# Get current figure to modifycurrent_fig = plt.gcf()current_ax = plt.gca()plt.tight_layout()plt.show()print('=' * 70)

In [ ]:
# Create bar plot for feature importancefig, ax = plt.subplots(figsize=(10, 8))# Calculate mean absolute SHAP valuesmean_shap = np.abs(shap_values_summary).mean(axis=0)sorted_idx = np.argsort(mean_shap)# Plotax.barh(range(len(sorted_idx)), mean_shap[sorted_idx],         color='steelblue', alpha=0.7, edgecolor='black')ax.set_yticks(range(len(sorted_idx)))ax.set_yticklabels(X_test.columns[sorted_idx], fontsize=10)ax.set_xlabel('Mean |SHAP value| (average impact on model output)',               fontweight='bold', fontsize=12)ax.set_title('📊 Feature Importance: Mean Absolute SHAP Values',             fontweight='bold', fontsize=14, pad=15)ax.grid(axis='x', alpha=0.3)plt.tight_layout()plt.show()# Print top featuresprint('\n🔝 TOP 5 MOST IMPORTANT FEATURES')print('=' * 70)top_features = sorted_idx[-5:][::-1]for i, idx in enumerate(top_features, 1):    feat = X_test.columns[idx]    importance = mean_shap[idx]    print(f'{i}. {feat:25} → {importance:.4f}')print('=' * 70)

In [ ]:
# Analyze feature distributionsprint('\n🔍 FEATURE DISTRIBUTION ANALYSIS')print('=' * 70)# Get top 3 featurestop_3 = sorted_idx[-3:][::-1]fig, axes = plt.subplots(1, 3, figsize=(18, 5))for i, feat_idx in enumerate(top_3):    ax = axes[i]    feat_name = X_test.columns[feat_idx]    feat_values = X_test.iloc[:, feat_idx].values    feat_shap = shap_values_summary[:, feat_idx]        # Scatter plot    scatter = ax.scatter(feat_values, feat_shap, c=feat_values,                         cmap='coolwarm', alpha=0.6, s=20, edgecolors='black', linewidth=0.5)        # Add trend line    z = np.polyfit(feat_values, feat_shap, 2)    p = np.poly1d(z)    x_smooth = np.linspace(feat_values.min(), feat_values.max(), 100)    ax.plot(x_smooth, p(x_smooth), 'r--', linewidth=2, label='Trend')        ax.set_xlabel(feat_name, fontweight='bold', fontsize=11)    ax.set_ylabel('SHAP value', fontweight='bold', fontsize=11)    ax.set_title(f'Feature: {feat_name}', fontweight='bold', fontsize=12)    ax.grid(alpha=0.3)    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)    ax.legend()        plt.colorbar(scatter, ax=ax, label='Feature value')plt.suptitle('🎨 Top 3 Features: Value vs SHAP Impact',              fontweight='bold', fontsize=16, y=1.02)plt.tight_layout()plt.show()print('\n💡 Interpretation Guide:')print('   • Positive slope: Higher values → Higher prediction')print('   • Negative slope: Higher values → Lower prediction')print('   • Curved line: Non-linear relationship')print('   • Scatter: Interaction effects with other features')print('=' * 70)

### ✏️ Your Turn: Exercise 11a**Task**: Compare global importance across different models1. Train 3 different models on the same data:   - Random Forest   - Gradient Boosting   - Logistic Regression2. Calculate SHAP values for each3. Create summary plots4. Compare:   - Do all models agree on top features?   - Which model shows strongest non-linearities?   - Are there model-specific patterns?5. What does this tell you about model selection?

In [ ]:
# YOUR CODE HERE# TODO: Compare feature importance across modelsmodels_to_compare = {    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE),    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=RANDOM_STATE),    'Logistic Regression': LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)}# Train each model# Calculate SHAP values# Compare importance rankings# Analyze differences

---## Exercise 12: Dependence Plots - Understanding Feature Effects### 📖 Concept**Dependence plots** show the relationship between a feature and its SHAP values.**Key questions answered**:1. How does feature X affect predictions?2. Is the relationship linear or non-linear?3. Does the effect depend on other features? (interaction coloring)**Elements**:- **X-axis**: Feature values- **Y-axis**: SHAP values (impact on prediction)- **Color**: Another feature (to show interactions)- **Scatter**: Individual instances

In [ ]:
# Create dependence plots for top featuresprint('🔗 DEPENDENCE PLOT ANALYSIS')print('=' * 70)# Use built-in SHAP dependence plottop_feature = X_test.columns[sorted_idx[-1]]print(f'Creating dependence plot for: {top_feature}')shap.dependence_plot(    top_feature,    shap_values_summary,    X_test,    interaction_index="auto",  # Automatically find best interaction    show=False)plt.title(f'🔗 Dependence Plot: {top_feature}', fontweight='bold', pad=15)plt.tight_layout()plt.show()print(f'\n💡 Auto-detected interaction feature shown in color')print('=' * 70)

In [ ]:
# Create manual dependence plot with custom interactionfig, axes = plt.subplots(2, 2, figsize=(16, 12))axes = axes.ravel()# Plot top 4 featuresfor i, feat_idx in enumerate(sorted_idx[-4:][::-1]):    ax = axes[i]    feat_name = X_test.columns[feat_idx]        # Get feature values and SHAP values    feat_vals = X_test.iloc[:, feat_idx].values    feat_shap = shap_values_summary[:, feat_idx]        # Find best interaction (highest correlation)    other_features = [j for j in range(len(X_test.columns)) if j != feat_idx]    correlations = []    for other_idx in other_features:        # Correlation between SHAP values and other feature        corr = np.corrcoef(feat_shap, X_test.iloc[:, other_idx])[0, 1]        correlations.append((other_idx, abs(corr)))        best_interaction_idx = max(correlations, key=lambda x: x[1])[0]    interaction_name = X_test.columns[best_interaction_idx]    interaction_vals = X_test.iloc[:, best_interaction_idx].values        # Create scatter plot    scatter = ax.scatter(feat_vals, feat_shap, c=interaction_vals,                        cmap='viridis', alpha=0.6, s=30, edgecolors='black', linewidth=0.5)        # Add regression line    z = np.polyfit(feat_vals, feat_shap, 2)    p = np.poly1d(z)    x_line = np.linspace(feat_vals.min(), feat_vals.max(), 100)    ax.plot(x_line, p(x_line), 'r--', linewidth=2, alpha=0.8, label='Trend')        ax.set_xlabel(feat_name, fontweight='bold', fontsize=11)    ax.set_ylabel('SHAP value', fontweight='bold', fontsize=11)    ax.set_title(f'{feat_name}\n(colored by {interaction_name})',                 fontweight='bold', fontsize=12)    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8, alpha=0.5)    ax.grid(alpha=0.3)    ax.legend()        # Add colorbar    cbar = plt.colorbar(scatter, ax=ax)    cbar.set_label(interaction_name, rotation=270, labelpad=15)plt.suptitle('🎨 Dependence Plots with Automatic Interaction Detection',             fontweight='bold', fontsize=16, y=1.0)plt.tight_layout()plt.show()print('\n💡 Reading dependence plots:')print('   • Flat line: Feature has no effect')print('   • Upward slope: Positive correlation')print('   • Downward slope: Negative correlation')print('   • Curve: Non-linear relationship')print('   • Color gradient: Interaction effect')print('=' * 70)

### ✏️ Your Turn: Exercise 12a**Task**: Deep dive into a specific featureChoose one interesting feature from the summary plot and:1. Create its dependence plot2. Identify the interaction feature (color)3. Divide data into quartiles based on the interaction feature4. Create 4 separate dependence plots (one per quartile)5. Describe how the relationship changes across quartiles6. Suggest a feature engineering strategy based on findings

In [ ]:
# YOUR CODE HERE# TODO: Deep dive into feature interactions# Select a feature to analyzefeature_to_analyze = X_test.columns[sorted_idx[-1]]# Create dependence plot# Find interaction feature# Split by interaction quartiles# Create 4 plots showing relationship in each quartile# Analyze and interpret

---## Exercise 13: Decision Plots - Comparing Multiple Instances### 📖 Concept**Decision plots** show the cumulative effect of features for multiple instances.**Perfect for**:- Comparing similar instances with different predictions- Understanding why two loans got different decisions- Debugging edge cases**Elements**:- **Y-axis**: Features (in order)- **X-axis**: Cumulative SHAP value- **Lines**: Different instances- **Base**: Expected value → Final prediction

In [ ]:
# Create decision plot for contrasting instancesprint('📊 DECISION PLOT: COMPARING INSTANCES')print('=' * 70)# Find two contrasting instances: one approved, one rejectedapproved_idx = X_test[y_test == 1].index[0]rejected_idx = X_test[y_test == 0].index[0]# Get their positions in X_testapproved_pos = X_test.index.get_loc(approved_idx)rejected_pos = X_test.index.get_loc(rejected_idx)print(f'Comparing two instances:')print(f'  Instance A (Approved): index {approved_pos}')print(f'  Instance B (Rejected): index {rejected_pos}')# Get SHAP values for these instancescomparison_shap = shap_values_summary[[approved_pos, rejected_pos], :]# Show feature valuesprint('\nFeature Values Comparison:')print('-' * 70)print(f'{"Feature":<25} {"Approved":<15} {"Rejected":<15} {"Difference"}')print('-' * 70)for feat in X_test.columns:    val_approved = X_test.loc[approved_idx, feat]    val_rejected = X_test.loc[rejected_idx, feat]    diff = val_approved - val_rejected    print(f'{feat:<25} {val_approved:<15.2f} {val_rejected:<15.2f} {diff:+.2f}')print('=' * 70)

In [ ]:
# Create custom decision plotfig, ax = plt.subplots(figsize=(12, 10))# Calculate cumulative SHAP valuesbase = explainer_shap.expected_valueif isinstance(base, list):    base = base[1]# Sort features by importance for approved instancefeat_importance = np.abs(comparison_shap[0])sorted_feat_idx = np.argsort(feat_importance)# Prepare datafeatures_ordered = X_test.columns[sorted_feat_idx]approved_cumsum = np.cumsum(comparison_shap[0][sorted_feat_idx])rejected_cumsum = np.cumsum(comparison_shap[1][sorted_feat_idx])# Add base valueapproved_full = np.concatenate([[base], base + approved_cumsum])rejected_full = np.concatenate([[base], base + rejected_cumsum])y_positions = range(len(features_ordered) + 1)# Plot linesax.plot(approved_full, y_positions, 'g-', linewidth=2.5, marker='o',         markersize=6, label='Approved ✅', alpha=0.8)ax.plot(rejected_full, y_positions, 'r-', linewidth=2.5, marker='o',        markersize=6, label='Rejected ❌', alpha=0.8)# Add feature labelsax.set_yticks(y_positions)ax.set_yticklabels(['Base value'] + features_ordered.tolist(), fontsize=10)# Styleax.set_xlabel('Cumulative SHAP value → Prediction', fontweight='bold', fontsize=12)ax.set_title('🎯 Decision Plot: How Features Accumulate to Final Prediction',             fontweight='bold', fontsize=14, pad=20)ax.axvline(x=0.5, color='black', linestyle='--', linewidth=1, alpha=0.5, label='Decision boundary')ax.grid(axis='x', alpha=0.3)ax.legend(loc='best', fontsize=11)# Add final predictions as textax.text(approved_full[-1], len(y_positions)-1, f' {approved_full[-1]:.3f}',         va='center', fontweight='bold', color='green', fontsize=10)ax.text(rejected_full[-1], len(y_positions)-1, f' {rejected_full[-1]:.3f}',        va='center', fontweight='bold', color='red', fontsize=10)plt.tight_layout()plt.show()print('\n💡 Interpretation:')print('   • Lines show cumulative effect of features')print('   • Divergence points show where decisions differ')print('   • Final positions are predicted probabilities')print('   • Features contributing most have biggest slopes')print('=' * 70)

### ✏️ Your Turn: Exercise 13a**Task**: Find and explain the most controversial decision1. Find instances where the model was very confident but WRONG2. Compare them with correctly classified similar instances3. Use decision plots to show:   - Where the predictions diverge   - Which features led to the wrong decision4. Propose ways to improve the model based on findings

In [ ]:
# YOUR CODE HERE# TODO: Analyze controversial/wrong predictions# Find misclassified instances with high confidencepredictions = rf_credit.predict_proba(X_test)[:, 1]misclassified = X_test[rf_credit.predict(X_test) != y_test]# Sort by confidence# Find similar correct predictions# Compare with decision plots

---# 🧠 Part 4: Advanced XAI Techniques for Deep Learning## Beyond SHAP: Specialized Methods for Neural NetworksWhile SHAP is powerful and general, deep learning has inspired specialized explanation methods:| Method | Type | Best For ||--------|------|----------|| **Attention** | Architecture-based | Transformers, seq2seq models || **Saliency Maps** | Gradient-based | CNNs, image classification || **Integrated Gradients** | Gradient-based | Any differentiable model || **GradCAM** | Gradient + Activation | CNNs, localization || **Layer-wise Relevance** | Backpropagation | Deep networks |### 🎯 When to Use What```Image Classification → GradCAM or Saliency MapsText/NLP → Attention MechanismsTabular Deep Learning → SHAP or Integrated GradientsTime Series → Attention + SHAPMulti-modal → Combination of methods```---

## Exercise 14: Attention as Explanation### 📖 Concept**Attention mechanisms** in neural networks naturally provide interpretability.**Key idea**:- Model learns to "focus" on important inputs- Attention weights show what the model pays attention to- High attention = high importance (usually)**Applications**:- 📝 **NLP**: Which words matter for sentiment?- 🖼️ **Vision**: Which image regions are relevant?- ⏰ **Time series**: Which past timesteps influence prediction?**Caveat**: Attention weights ≠ explanations (but strong correlation)

In [ ]:
# Simulate attention mechanismprint('🔍 ATTENTION MECHANISM SIMULATION')print('=' * 70)# Create synthetic time series datanp.random.seed(RANDOM_STATE)n_samples = 100sequence_length = 20# Generate sequences where last 5 timesteps are most importantX_seq = np.random.randn(n_samples, sequence_length)# Add signal to last 5 timestepsX_seq[:, -5:] += np.random.randn(n_samples, 5) * 2# Target depends mainly on last 5 timestepsy_seq = np.sum(X_seq[:, -5:], axis=1)y_seq = (y_seq > 0).astype(int)print(f'Generated sequential data:')print(f'  Samples: {n_samples}')print(f'  Sequence length: {sequence_length}')print(f'  Important timesteps: last 5')print(f'  Target distribution: {np.mean(y_seq):.2%} positive')# Simple attention simulation using correlationattention_weights = np.zeros(sequence_length)for t in range(sequence_length):    # Attention = correlation with target    correlation = np.corrcoef(X_seq[:, t], y_seq)[0, 1]    attention_weights[t] = abs(correlation)# Normalizeattention_weights = attention_weights / attention_weights.sum()print('\n✅ Attention weights computed (based on correlation)')print('=' * 70)

In [ ]:
# Visualize attention weightsfig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))# Top: Attention weights over timetimesteps = range(1, sequence_length + 1)bars = ax1.bar(timesteps, attention_weights, color='skyblue',                alpha=0.7, edgecolor='black', linewidth=1.5)# Highlight important regionfor i in range(sequence_length - 5, sequence_length):    bars[i].set_color('coral')    bars[i].set_alpha(0.9)ax1.set_xlabel('Timestep', fontweight='bold', fontsize=12)ax1.set_ylabel('Attention Weight', fontweight='bold', fontsize=12)ax1.set_title('🎯 Attention Weights Over Sequence', fontweight='bold', fontsize=14)ax1.grid(axis='y', alpha=0.3)ax1.legend(['Early timesteps', 'Important timesteps (last 5)'], loc='upper left')# Bottom: Heatmap of sample sequences with attentionn_display = 10display_data = X_seq[:n_display]im = ax2.imshow(display_data, aspect='auto', cmap='RdBu_r', vmin=-3, vmax=3)# Add attention overlayfor t in range(sequence_length):    for s in range(n_display):        alpha = attention_weights[t] * 3  # Scale for visibility        ax2.add_patch(plt.Rectangle((t-0.5, s-0.5), 1, 1,                                     fill=False, edgecolor='yellow',                                     linewidth=alpha*5, alpha=min(alpha, 1)))ax2.set_xlabel('Timestep', fontweight='bold', fontsize=12)ax2.set_ylabel('Sample', fontweight='bold', fontsize=12)ax2.set_title('📊 Sequence Data with Attention Overlay (yellow=high attention)',              fontweight='bold', fontsize=14)plt.colorbar(im, ax=ax2, label='Value')plt.tight_layout()plt.show()print('\n💡 Interpretation:')print('   • Attention correctly focuses on last 5 timesteps')print('   • Yellow borders show where model "looks"')print('   • This is how Transformers and RNNs explain themselves')print('=' * 70)

### ✏️ Your Turn: Exercise 14a**Task**: Implement attention for text sentiment analysis1. Create synthetic "sentences" (sequences of word embeddings)2. Make some positions more predictive of sentiment3. Calculate attention weights4. Visualize which "words" the model attends to5. Compare with SHAP values on the same data

In [ ]:
# YOUR CODE HERE# TODO: Attention mechanism for text# Create synthetic word sequencesvocab_size = 100embedding_dim = 10sentence_length = 15# Generate data where certain positions indicate sentiment# Calculate attention weights# Compare with SHAP

---## Exercise 15: Saliency Maps and Integrated Gradients### 📖 Concept**Saliency Maps** use gradients to identify important input features.**Core formula**:```Saliency(x) = |∂f/∂x|```**Integrated Gradients** improve this:```IG(x) = (x - x') × ∫₀¹ ∂f/∂x̄ dα```Where:- x = input- x' = baseline (typically zeros)- Integral computed over path from x' to x**Advantages**:- ⚡ Fast (single backward pass)- 🎯 Highlights input regions- ✅ Satisfies axioms (completeness, sensitivity)

In [ ]:
# Simulate gradient-based attributionprint('🎨 GRADIENT-BASED ATTRIBUTION SIMULATION')print('=' * 70)# Create synthetic image-like data (simplified)np.random.seed(RANDOM_STATE)img_size = 28n_images = 100# Generate images with a pattern in the centerimages = np.random.randn(n_images, img_size, img_size) * 0.1# Add signal in center regioncenter = img_size // 2radius = 5for i in range(n_images):    # Create pattern in center    for y in range(center-radius, center+radius):        for x in range(center-radius, center+radius):            if (y-center)**2 + (x-center)**2 <= radius**2:                images[i, y, x] += np.random.randn() * 2# Target based on center regiony_img = (images[:, center-radius:center+radius, center-radius:center+radius].mean(axis=(1,2)) > 0).astype(int)print(f'Generated image data:')print(f'  Samples: {n_images}')print(f'  Image size: {img_size}x{img_size}')print(f'  Important region: Center ({radius*2}x{radius*2})')print(f'  Target distribution: {np.mean(y_img):.2%} positive')# Simulate gradient-based saliency# In real CNNs: saliency = |∂L/∂x|# Here: simulate with importance map# Calculate correlation of each pixel with targetsaliency_map = np.zeros((img_size, img_size))for y in range(img_size):    for x in range(img_size):        pixel_values = images[:, y, x]        correlation = np.corrcoef(pixel_values, y_img)[0, 1]        saliency_map[y, x] = abs(correlation)# Normalizesaliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min())print('\n✅ Saliency map computed')print('=' * 70)

In [ ]:
# Visualize saliency mapsfig, axes = plt.subplots(2, 3, figsize=(15, 10))# Top row: Sample imagesfor i in range(3):    ax = axes[0, i]    ax.imshow(images[i], cmap='gray')    ax.set_title(f'Image {i+1}\nLabel: {"+" if y_img[i] else "-"}',                  fontweight='bold')    ax.axis('off')# Bottom row: Saliency maps overlaidfor i in range(3):    ax = axes[1, i]        # Show image in grayscale    ax.imshow(images[i], cmap='gray', alpha=0.5)        # Overlay saliency map    saliency_overlay = ax.imshow(saliency_map, cmap='hot', alpha=0.6, vmin=0, vmax=1)        ax.set_title(f'Saliency Map {i+1}', fontweight='bold')    ax.axis('off')# Add colorbarcbar = plt.colorbar(saliency_overlay, ax=axes[1, :], orientation='horizontal',                    fraction=0.046, pad=0.04)cbar.set_label('Saliency (Higher = More Important)', fontweight='bold')plt.suptitle('🎨 Gradient-based Saliency Maps', fontweight='bold', fontsize=16, y=0.98)plt.tight_layout()plt.show()print('\n💡 Interpretation:')print('   • Bright regions (yellow/white) are most important')print('   • Center region correctly identified as important')print('   • This is how CNNs explain image classifications')print('=' * 70)

In [ ]:
# Integrated Gradients simulationprint('\n🔬 INTEGRATED GRADIENTS SIMULATION')print('=' * 70)# For a single image, compute integrated gradientsample_img = images[0]baseline = np.zeros_like(sample_img)# Steps along pathn_steps = 50alphas = np.linspace(0, 1, n_steps)# Interpolated imagesinterpolated = np.array([baseline + alpha * (sample_img - baseline) for alpha in alphas])print(f'Computing integrated gradient:')print(f'  Steps: {n_steps}')print(f'  Path: baseline (zeros) → actual image')# Simulate gradient at each step (using correlation with target as proxy)integrated_grad = np.zeros_like(sample_img)for y in range(img_size):    for x in range(img_size):        # Average gradient along path        grads = []        for step_img in interpolated:            # Simulate: gradient ≈ importance            grad = saliency_map[y, x]            grads.append(grad)                # Integrated gradient = (x - baseline) * average gradient        integrated_grad[y, x] = (sample_img[y, x] - baseline[y, x]) * np.mean(grads)# Normalizeintegrated_grad = np.abs(integrated_grad)integrated_grad = (integrated_grad - integrated_grad.min()) / (integrated_grad.max() - integrated_grad.min() + 1e-10)print('\n✅ Integrated gradients computed')print('=' * 70)# Visualizefig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))# Original imageax1.imshow(sample_img, cmap='gray')ax1.set_title('Original Image', fontweight='bold')ax1.axis('off')# Saliency mapax2.imshow(sample_img, cmap='gray', alpha=0.5)im2 = ax2.imshow(saliency_map, cmap='hot', alpha=0.6)ax2.set_title('Saliency Map', fontweight='bold')ax2.axis('off')plt.colorbar(im2, ax=ax2, fraction=0.046)# Integrated gradientsax3.imshow(sample_img, cmap='gray', alpha=0.5)im3 = ax3.imshow(integrated_grad, cmap='hot', alpha=0.6)ax3.set_title('Integrated Gradients', fontweight='bold')ax3.axis('off')plt.colorbar(im3, ax=ax3, fraction=0.046)plt.suptitle('🔬 Comparison: Saliency vs Integrated Gradients',              fontweight='bold', fontsize=16, y=1.02)plt.tight_layout()plt.show()print('\n💡 Key Difference:')print('   • Saliency: Single gradient at input')print('   • Integrated Gradients: Average gradient along path')print('   • IG is more stable and satisfies axioms')print('=' * 70)

### ✏️ Your Turn: Exercise 15a**Task**: Apply gradient methods to real dataUsing our credit approval model:1. Calculate feature gradients for a specific prediction2. Compare with SHAP values3. Analyze:   - Do gradients and SHAP agree on important features?   - What are the pros/cons of each method?   - When would you use gradients over SHAP?4. For non-differentiable features (like categorical), how would you adapt?

In [ ]:
# YOUR CODE HERE# TODO: Apply gradient methods to credit data# Hint: For tree models, approximate gradients using:# - Small perturbations (numerical gradient)# - Local linear approximation# Compare with TreeSHAP

---## Exercise 16: GradCAM - Where Does the Model Look?### 📖 Concept**GradCAM** (Gradient-weighted Class Activation Mapping) localizes important regions in images.**How it works**:1. Forward pass through network2. Backward pass to get gradients3. Weight feature maps by gradients4. Generate heatmap of important regions**Formula**:```L_GradCAM = ReLU(Σ αk × Ak)where αk = (1/Z) × Σ(∂y/∂Ak)```**Evolution**:- CAM (2016): Requires GAP layer- GradCAM (2017): Works with any CNN- GradCAM++ (2018): Better for multiple objects- ScoreCAM (2020): No gradients needed

In [ ]:
# Simulate GradCAM conceptprint('🎯 GRADCAM SIMULATION')print('=' * 70)# Use our synthetic images from before# Simulate conv layer activation maps# Create "feature maps" (simplified)n_feature_maps = 5feature_maps = []for f in range(n_feature_maps):    # Each feature map detects different pattern    fmap = np.random.randn(img_size//2, img_size//2)        # Make one feature map focus on center    if f == 2:  # "center detector"        center_half = (img_size//2) // 2        fmap[center_half-3:center_half+3, center_half-3:center_half+3] += 2        feature_maps.append(fmap)# Simulate gradients (importance of each feature map)gradients = np.array([0.2, 0.3, 0.8, 0.1, 0.2])  # Feature map 2 is most importantprint(f'Simulated CNN layers:')print(f'  Feature maps: {n_feature_maps}')print(f'  Feature map size: {img_size//2}x{img_size//2}')print(f'  Gradients (importance): {gradients}')# Compute GradCAMgradcam = np.zeros((img_size//2, img_size//2))for f in range(n_feature_maps):    gradcam += gradients[f] * feature_maps[f]# Apply ReLUgradcam = np.maximum(gradcam, 0)# Normalizegradcam = (gradcam - gradcam.min()) / (gradcam.max() - gradcam.min())# Upsample to original sizefrom scipy import ndimagegradcam_upsampled = ndimage.zoom(gradcam, 2, order=1)print('\n✅ GradCAM heatmap generated')print('=' * 70)

In [ ]:
# Visualize GradCAMfig, axes = plt.subplots(2, 3, figsize=(15, 10))# Top row: Feature mapsfor i in range(3):    ax = axes[0, i]    im = ax.imshow(feature_maps[i], cmap='viridis')    ax.set_title(f'Feature Map {i+1}\nImportance: {gradients[i]:.2f}',                 fontweight='bold')    ax.axis('off')    plt.colorbar(im, ax=ax, fraction=0.046)# Bottom left: Original imageaxes[1, 0].imshow(images[0], cmap='gray')axes[1, 0].set_title('Original Image', fontweight='bold')axes[1, 0].axis('off')# Bottom middle: GradCAM heatmapim = axes[1, 1].imshow(gradcam_upsampled, cmap='jet')axes[1, 1].set_title('GradCAM Heatmap', fontweight='bold')axes[1, 1].axis('off')plt.colorbar(im, ax=axes[1, 1], fraction=0.046)# Bottom right: Overlayaxes[1, 2].imshow(images[0], cmap='gray', alpha=0.7)axes[1, 2].imshow(gradcam_upsampled, cmap='jet', alpha=0.5)axes[1, 2].set_title('GradCAM Overlay', fontweight='bold')axes[1, 2].axis('off')plt.suptitle('🎯 GradCAM: Visual Explanation for CNN',              fontweight='bold', fontsize=16, y=0.98)plt.tight_layout()plt.show()print('\n💡 Interpretation:')print('   • Red/yellow regions: Where model focuses')print('   • Blue regions: Less important')print('   • Overlay shows important areas on original image')print('   • Useful for debugging: Is model looking at right place?')print('=' * 70)

### ✏️ Your Turn: Exercise 16a**Task**: Understand GradCAM applicationsResearch and summarize:1. **Medical Imaging**: How is GradCAM used to explain disease diagnosis?2. **Autonomous Driving**: How does it help understand what the car "sees"?3. **Quality Control**: Applications in manufacturing defect detection4. **Limitations**: When does GradCAM fail or mislead?5. **Alternatives**: Compare with other visualization methods (Saliency, LIME for images)Prepare a short presentation (5 slides) on one application.

In [ ]:
# YOUR RESEARCH HERE# TODO: Research GradCAM applications# Ideas:# - Find examples online# - Try implementing on a pre-trained model# - Compare with other methods# - Document use cases# Present findings as markdown

---# 🎯 Part 5: Real-World Application & Best Practices## Exercise 17: End-to-End Explainability Pipeline### 📖 Scenario: Credit Approval SystemYou're deploying a credit approval model. **Requirements**:1. **Accuracy**: Model must perform well2. **Explainability**: Every decision must be explainable3. **Fairness**: Check for discriminatory patterns4. **Compliance**: Meet regulatory requirements5. **Usability**: Non-technical staff should understandLet's build a complete explainability pipeline!

In [ ]:
# Complete explainability pipelineprint('🏗️ BUILDING COMPLETE XAI PIPELINE')print('=' * 70)# Step 1: Train modelprint('\n📊 Step 1: Model Training')print('-' * 70)X = credit_df.drop('Approved', axis=1)y = credit_df['Approved']X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)# Train final modelfinal_model = RandomForestClassifier(n_estimators=200, max_depth=8,                                      min_samples_leaf=10, random_state=RANDOM_STATE)final_model.fit(X_train_full, y_train_full)train_acc = final_model.score(X_train_full, y_train_full)test_acc = final_model.score(X_test_full, y_test_full)print(f'✅ Model trained')print(f'   Training accuracy: {train_acc:.3f}')print(f'   Test accuracy: {test_acc:.3f}')# Step 2: Global explainabilityprint('\n🌍 Step 2: Global Explainability Analysis')print('-' * 70)explainer_final = shap.TreeExplainer(final_model)shap_values_final = explainer_final.shap_values(X_test_full)if isinstance(shap_values_final, list):    shap_values_final = shap_values_final[1]# Feature importancemean_abs_shap_final = np.abs(shap_values_final).mean(axis=0)feature_importance_df = pd.DataFrame({    'Feature': X.columns,    'Importance': mean_abs_shap_final}).sort_values('Importance', ascending=False)print('\n🔝 Top 5 Most Important Features:')for idx, row in feature_importance_df.head(5).iterrows():    print(f'   {row["Feature"]:25} → {row["Importance"]:.4f}')print('=' * 70)

In [ ]:
# Step 3: Create explanation functionprint('\n🔍 Step 3: Individual Explanation Function')print('-' * 70)def explain_prediction(instance_idx, model, explainer, X_data, y_data):    """    Generate comprehensive explanation for a single prediction.        Returns: dict with explanation components    """    # Get instance    instance = X_data.iloc[instance_idx:instance_idx+1]    actual_label = y_data.iloc[instance_idx]        # Prediction    prediction_proba = model.predict_proba(instance)[0][1]    prediction_label = 1 if prediction_proba >= 0.5 else 0        # SHAP values    shap_vals = explainer.shap_values(instance)    if isinstance(shap_vals, list):        shap_vals = shap_vals[1][0]    else:        shap_vals = shap_vals[0]        # Base value    base_value = explainer.expected_value    if isinstance(base_value, list):        base_value = base_value[1]        # Feature contributions    contributions = pd.DataFrame({        'Feature': X_data.columns,        'Value': instance.values[0],        'SHAP': shap_vals,        'Abs_SHAP': np.abs(shap_vals)    }).sort_values('Abs_SHAP', ascending=False)        return {        'instance_idx': instance_idx,        'prediction': prediction_label,        'probability': prediction_proba,        'actual': actual_label,        'correct': prediction_label == actual_label,        'base_value': base_value,        'contributions': contributions,        'shap_values': shap_vals    }# Test the functiontest_idx = 42explanation = explain_prediction(test_idx, final_model, explainer_final,                                 X_test_full, y_test_full)print(f'✅ Explanation generated for instance {test_idx}')print(f'\n📋 Prediction Report:')print(f'   Actual: {"Approved" if explanation["actual"] else "Rejected"}')print(f'   Predicted: {"Approved" if explanation["prediction"] else "Rejected"}')print(f'   Probability: {explanation["probability"]:.1%}')print(f'   Correct: {explanation["correct"]}')print(f'\n🎯 Top 3 Contributing Features:')for idx, row in explanation['contributions'].head(3).iterrows():    direction = "↑ Increases" if row['SHAP'] > 0 else "↓ Decreases"    print(f'   • {row["Feature"]:20} = {row["Value"]:7.2f} → {direction} approval by {abs(row["SHAP"]):.3f}')print('=' * 70)

In [ ]:
# Step 4: Batch explanation for rejected applicationsprint('\n❌ Step 4: Analyzing Rejected Applications')print('=' * 70)# Find rejected predictionspredictions_full = final_model.predict(X_test_full)rejected_mask = predictions_full == 0rejected_indices = X_test_full[rejected_mask].indexprint(f'Rejected applications: {rejected_mask.sum()} out of {len(X_test_full)}')# Analyze top reasons for rejectionrejected_shap = shap_values_final[rejected_mask]mean_rejected_contributions = np.abs(rejected_shap).mean(axis=0)rejection_reasons = pd.DataFrame({    'Feature': X.columns,    'Avg_Impact': mean_rejected_contributions}).sort_values('Avg_Impact', ascending=False)print('\n🔝 Top Rejection Factors:')for idx, row in rejection_reasons.head(5).iterrows():    print(f'   {idx+1}. {row["Feature"]:25} → Avg impact: {row["Avg_Impact"]:.4f}')# Find common patternsprint('\n📊 Common Patterns in Rejected Applications:')for feat in rejection_reasons.head(3)['Feature']:    feat_values_rejected = X_test_full.loc[rejected_indices, feat]    print(f'   {feat:25}: Mean={feat_values_rejected.mean():.2f}, Std={feat_values_rejected.std():.2f}')print('=' * 70)

In [ ]:
# Step 5: Create comprehensive reportprint('\n📄 Step 5: Generate Explanation Report')print('=' * 70)# Select diverse examplessample_approved = X_test_full[(predictions_full == 1) & (y_test_full == 1)].index[0]sample_rejected = X_test_full[(predictions_full == 0) & (y_test_full == 0)].index[0]sample_false_positive = X_test_full[(predictions_full == 1) & (y_test_full == 0)].index[0] if any((predictions_full == 1) & (y_test_full == 0)) else Nonesample_false_negative = X_test_full[(predictions_full == 0) & (y_test_full == 1)].index[0] if any((predictions_full == 0) & (y_test_full == 1)) else Nonesamples_to_explain = {    'Correct Approval': sample_approved,    'Correct Rejection': sample_rejected,    'False Positive': sample_false_positive,    'False Negative': sample_false_negative}# Generate explanationsfig, axes = plt.subplots(2, 2, figsize=(16, 12))axes = axes.ravel()for idx, (case_name, sample_idx) in enumerate(samples_to_explain.items()):    if sample_idx is None:        axes[idx].text(0.5, 0.5, f'{case_name}\nNo examples found',                       ha='center', va='center', fontsize=14)        axes[idx].axis('off')        continue        ax = axes[idx]        # Get explanation    expl = explain_prediction(X_test_full.index.get_loc(sample_idx),                              final_model, explainer_final,                              X_test_full, y_test_full)        # Plot waterfall-style    top_feats = expl['contributions'].head(7)    y_pos = range(len(top_feats))    colors = ['green' if x > 0 else 'red' for x in top_feats['SHAP']]        ax.barh(y_pos, top_feats['SHAP'], color=colors, alpha=0.7, edgecolor='black')    ax.set_yticks(y_pos)    ax.set_yticklabels([f'{row["Feature"]}\n={row["Value"]:.1f}'                         for _, row in top_feats.iterrows()], fontsize=9)    ax.set_xlabel('SHAP Value (impact on approval)', fontweight='bold')    ax.axvline(x=0, color='black', linestyle='--', linewidth=1)    ax.grid(axis='x', alpha=0.3)        # Title with case info    status = "✅" if expl['correct'] else "❌"    title = f'{case_name} {status}\nProbability: {expl["probability"]:.1%}'    ax.set_title(title, fontweight='bold', fontsize=11)plt.suptitle('📊 Comprehensive Explanation Report: Diverse Cases',             fontweight='bold', fontsize=16, y=0.995)plt.tight_layout()plt.show()print('\n✅ Explanation report generated for all case types')print('=' * 70)

### 💡 Best Practices Summary#### 1. **Model Selection**- ✅ Choose TreeSHAP for tree-based models (fast & exact)- ✅ Use KernelSHAP for black-box models (slower but universal)- ✅ Consider interpretable models (logistic regression, decision trees) when possible#### 2. **Explanation Quality**- ✅ Verify explanations make domain sense- ✅ Check consistency across similar instances- ✅ Compare with feature importance from other methods#### 3. **Communication**- ✅ Use waterfall plots for individual decisions- ✅ Use summary plots for overall model behavior- ✅ Tailor explanations to audience (technical vs non-technical)#### 4. **Fairness & Bias**- ✅ Check SHAP values across protected groups- ✅ Identify discriminatory patterns- ✅ Use explanations to debug bias#### 5. **Production Deployment**- ✅ Pre-compute explanations for common scenarios- ✅ Cache SHAP values for frequently queried instances- ✅ Monitor explanation drift over time- ✅ Provide explanation APIs alongside predictions#### 6. **Common Pitfalls to Avoid**- ❌ Don't rely solely on feature importance (use instance-level too)- ❌ Don't ignore base values in waterfall plots- ❌ Don't assume correlation = causation- ❌ Don't forget to validate explanations with domain experts- ❌ Don't use overly complex visualizations for stakeholders---

## 🎓 Final Summary### What We've Learned1. **SHAP Fundamentals** (Part 1)   - Shapley values from game theory   - Mathematical properties and guarantees   - SHAP vs LIME comparison2. **Implementation Methods** (Part 2)   - TreeSHAP for tree-based models   - KernelSHAP for any model   - DeepSHAP for neural networks   - GradientSHAP for differentiable models   - Interaction values3. **Visualizations** (Part 3)   - Waterfall plots for single predictions   - Summary plots for global importance   - Dependence plots for feature effects   - Decision plots for comparisons4. **Advanced Techniques** (Part 4)   - Attention mechanisms   - Saliency maps   - Integrated gradients   - GradCAM for CNNs5. **Real-World Application** (Part 5)   - End-to-end explainability pipeline   - Best practices   - Common pitfalls### 📚 Further Resources**Papers**:- Lundberg & Lee (2017): "A Unified Approach to Interpreting Model Predictions"- Ribeiro et al. (2016): "Why Should I Trust You?"- Selvaraju et al. (2017): "Grad-CAM"**Libraries**:- SHAP: https://github.com/slundberg/shap- LIME: https://github.com/marcotcr/lime- Captum (PyTorch): https://captum.ai/- Alibi: https://github.com/SeldonIO/alibi**Books**:- "Interpretable Machine Learning" by Christoph Molnar- "Explanatory Model Analysis" by Biecek & Burzykowski### ✏️ Final Exercise**Task**: Create Your Own XAI DashboardBuild an interactive explanation dashboard for any model:1. Train a model on a dataset of your choice2. Implement:   - Global feature importance   - Instance-level explanations   - What-if analysis (change features, see prediction change)   - Fairness analysis across groups3. Use Plotly or Streamlit for interactivity4. Add export functionality (PDF reports)5. Document:   - How to use the dashboard   - Interpretation guidelines   - Limitations and caveats**Deliverables**:- Python code- Sample report- 5-minute demo video---## 🎉 Congratulations!You've completed the comprehensive SHAP & XAI tutorial!**Next Steps**:1. Apply SHAP to your own projects2. Experiment with different explainers3. Contribute to open-source XAI tools4. Stay updated with latest research**Questions?** Contact: homin.park@ghent.ac.kr**Feedback?** Let us know how to improve this tutorial!---### 📝 Quick Reference Cheat Sheet```python# TreeSHAP (for Random Forest, XGBoost, etc.)explainer = shap.TreeExplainer(model)shap_values = explainer.shap_values(X)# KernelSHAP (for any model)explainer = shap.KernelExplainer(model.predict, background_data)shap_values = explainer.shap_values(X, nsamples=100)# Visualizationsshap.waterfall_plot(shap_values[0])  # Single instanceshap.summary_plot(shap_values, X)     # Global overviewshap.dependence_plot("feature", shap_values, X)  # Feature effect# Verify local accuracybase_value + shap_values.sum() ≈ model.predict(x)```**Remember**: Explainability is not just about algorithms—it's about communication!---*This tutorial was created with ❤️ for the AI/ML community. Please share and adapt as needed!*